In [ ]:
import numpy as np

import section3_1 as ld
from section3_1 import calc_etr_dash_t

from section4_1 import get_heating_mode, get_default_heating_spec, get_default_heatsource

import section9_3 as ass

# ダクト式セントラル空調機
import section4_2 as dc
import section4_2 as dc_spec

# エアーコンディショナー
import section4_3 as rac
import section4_3 as rac_spec

# FF暖房
import section4_4 as ff
import section4_4 as ff_spec

# 電気ヒーター床暖房
import section4_5 as eheater
import section4_5 as eheater_spec

# 電気蓄熱暖房
import section4_6 as ets
import section4_6 as ets_spec

# 温水暖房
import section4_7 as hwh

# 温水暖房用パネルラジエーター
import section4_7_j as rad_panel

# 温水暖房用ファンコンベクター
import section4_7_k as rad_fanc

# 温水暖房用床暖房
import section4_7_l as rad_floor

# ルームエアコンディショナー付温水床暖房
import section4_8 as racfh
import section4_8 as racfh_spec

# 第四章　暖冷房設備

## 第一節　全般

1．適用範囲～4．記号及び単位は省略

### 5．暖房方式及び冷房方式

省略

### 6．暖房設備の一次エネルギー消費量及び処理負荷と未処理負荷

#### 6.1 処理負荷及び未処理負荷

# 要確認：仕様書では数式はないが、プログラムでの計算上必要だと思われる以下のコードはどうすればよいのか

In [ ]:
# 暖房負荷の取得
def get_heating_load(region, sol_region, A_A, A_MR, A_OR, Q, mu_H, TS, r_A_ufvnt, HEX, underfloor_insulation, mode_H,
                     spec_MR, spec_OR, mode_MR, mode_OR, SHC):
    if mode_H == '住戸全体を連続的に暖房する方式':
        # 住宅全体を連続的に暖房する方式おける暖房負荷 (2)
        L_H_A_d_t = get_L_H_A_d_t(spec_MR, spec_OR, SHC, A_A, A_MR, A_OR, region, sol_region, Q, mu_H, TS,
                                  r_A_ufvnt,
                                  HEX, underfloor_insulation)
        return L_H_A_d_t, None, None
    elif mode_H == '居室のみを暖房する方式でかつ主たる居室とその他の居室ともに温水暖房を設置する場合に該当しない場合':
        # 暖房区画i=1-5それぞれの暖房負荷
        L_T_H_d_t_i, L_dash_H_R_d_t_i = get_L_H_d_t(region, sol_region, A_A, A_MR, A_OR, None, spec_MR, spec_OR,
                                                    mode_MR, mode_OR, Q,
                                                    mu_H, TS, r_A_ufvnt, HEX, SHC, underfloor_insulation)
        return None, L_T_H_d_t_i, L_dash_H_R_d_t_i
    elif mode_H is None:
        return None, None, None
    else:
        raise ValueError(mode_H)

#### 6.1.1 住戸全体を連続的に暖房する方式

$$
\large
Q_{T,H,A,d,t} = min(Q_{max,H,d,t}, L_{H,A,d,t})
$$

$$
\large
Q_{UT,H,A,d,t} = L_{H,A,d,t} - Q_{T,H,A,d,t}
$$

$Q_{T,H,A,d,t}$：住戸全体を連続的に暖房する方式における日付$d$の時刻$t$における1時間当たりの暖房設備の処理暖房負荷(MJ/h)  
$Q_{UT,H,A,d,t}$：住戸全体を連続的に暖房する方式における日付$d$の時刻$t$における1時間当たりの暖房設備の未処理暖房負荷(MJ/h)  
$Q_{max,H,d,t}$：日付$d$の時刻$t$における1時間当たりの暖房設備機器の最大暖房出力(MJ/h)  
$L_{H,A,d,t}$：住戸全体を連続的に暖房する方式における日付$d$の時刻$t$における1時間当たりの暖房負荷の合計(MJ/h)  

In [ ]:
# 住宅全体を連続的に暖房する方式おける処理暖房負荷 (1a)
def get_Q_T_H_A_d_t(Q_max_H_d_t, L_H_A_d_t):
    return np.min([Q_max_H_d_t, L_H_A_d_t], axis=0)

# 住宅全体を連続的に暖房する方式おける未処理暖房負荷 (1b)
def get_Q_UT_H_A_d_t(L_H_A_d_t, Q_T_H_A_d_t):
    return L_H_A_d_t - Q_T_H_A_d_t

$L_{H,A,d,t}$が0未満の場合は0とする。 

$$
\large
L_{H,A,d,t} = \sum_{i=1～12}L_{H,d,t,i} 
$$

In [ ]:
# 住宅全体を連続的に暖房する方式おける暖房負荷(2)
def get_L_H_A_d_t(H_MR, H_OR, SHC, A_A, A_MR, A_OR, region, sol_region, Q, mu_H, TS, r_A_ufvnt, HEX,
                  underfloor_insulation):
    L_H_d_t, _ = get_L_H_d_t(region, sol_region, A_A, A_MR, A_OR, '住戸全体を連続的に暖房する方式', H_MR, H_OR, None, None, Q, mu_H,
                             TS, r_A_ufvnt,
                             HEX, SHC, underfloor_insulation, normalize=False)
    L_H_A_d_t = np.sum(L_H_d_t, axis=0)
    L_H_A_d_t[L_H_A_d_t < 0] = 0
    return L_H_A_d_t

#### 6.1.2 居室のみを暖房する方式

$$
\large
Q_{T,H,MR,d,t} = Q_{T,H,d,t,i}|_{i=1}
$$

$$
\large
Q_{UT,H,MR,d,t} = Q_{UT,H,d,t,i}|_{i=1}
$$

$Q_{T,H,MR,d,t}$：居室のみを暖房する方式における日付$d$の時刻$t$における1時間当たりの主たる居室に設置された暖房設備の処理暖房負荷(MJ/h)  
$Q_{UT,H,MR,d,t}$：居室のみを暖房する方式における日付$d$の時刻$t$における1時間当たりの主たる居室に設置された暖房設備の未処理暖房負荷(MJ/h)  
$Q_{T,H,d,t,i}$：日付$d$の時刻$t$における1時間当たりの暖冷房区画𝑖に設置された暖房設備機器等の処理暖房負荷(MJ/h)  
$Q_{UT,H,d,t,i}$：日付$d$の時刻$t$における1時間当たりの暖冷房区画𝑖に設置された暖房設備機器等の未処理暖房負荷(MJ/h)  

# 要確認：(3a)がコメントアウトしている？

In [ ]:
# # 主たる居室に設置された暖房設備の処理暖房負荷 (3a)
# def get_Q_T_H_MR_d_t():
#     return get_Q_T_H_d_t_i(i=1)

# 主たる居室に設置された暖房設備の未処理暖房負荷 (3b)
def get_Q_UT_H_MR_d_t(region, A_A, A_MR, A_OR, spec_MR, spec_OR, spec_HS, mode_MR, mode_OR, CG, L_T_H_d_t):
    if spec_MR['type'] in ['温水暖房用パネルラジエーター', '温水暖房用床暖房', '温水暖房用ファンコンベクター']:

        # 送水温度の決定
        Theta_SW_hs_op = hwh.get_Theta_SW_hs_op(spec_HS['type'], CG)
        rad_list = hwh.get_rad_list(spec_MR, spec_OR)
        p_hs_d_t = hwh.get_p_hs_d_t(Theta_SW_hs_op, rad_list, L_T_H_d_t, A_A, A_MR, A_OR, region, mode_MR, mode_OR)
        Theta_SW_d_t = hwh.get_Theta_SW_d_t(Theta_SW_hs_op, p_hs_d_t)

        if spec_MR['type'] == '温水暖房用パネルラジエーター':
            # 床面積
            A_HCZ = get_A_HCZ_i(1, A_A, A_MR, A_OR)
            R_type = '主たる居室'

            # パネルラジエーターの最大能力
            q_max_rad = rad_panel.get_q_max_rad(region, mode_MR, A_HCZ, R_type)
            # パネルラジエーターの最大暖房出力
            Q_max_H_rad = rad_panel.get_Q_max_H_rad(Theta_SW_d_t, q_max_rad)
        elif spec_MR['type'] == '温水暖房用床暖房':
            # 床面積
            A_HCZ = get_A_HCZ_i(1, A_A, A_MR, A_OR)
            r_Af = spec_MR['r_Af']
            A_f = rad_floor.get_A_f(A_HCZ, r_Af)
            # 温水床暖房の単位面積当たりの上面最大放熱能力
            Q_max_H_rad = rad_floor.get_Q_max_H_rad(Theta_SW_d_t, A_f)
        elif spec_MR['type'] == '温水暖房用ファンコンベクター':
            # 床面積
            A_HCZ = get_A_HCZ_i(1, A_A, A_MR, A_OR)
            R_type = '主たる居室'
            q_max_FC = rad_fanc.get_q_max_FC(region, mode_MR, A_HCZ, R_type)
            Q_max_H_rad = rad_fanc.get_Q_max_H_rad(Theta_SW_d_t, q_max_FC)
        else:
            raise ValueError(spec_MR['type'])

        # 処理負荷
        Q_T_H_d_t_i = np.min([Q_max_H_rad, L_T_H_d_t[0]], axis=0)

        # 未処理負荷
        Q_UT_H_d_t_i = L_T_H_d_t[0] - Q_T_H_d_t_i

        print('{} Q_UT_H_d_t_1 = {} [MJ]'.format(spec_MR['type'], np.sum(Q_UT_H_d_t_i)))

        return Q_UT_H_d_t_i
    else:
        return calc_Q_UT_H_d_t(1, spec_MR, A_A, A_MR, A_OR, region, mode_MR, L_T_H_d_t[0])

$$
\large
Q_{T,H,OR,d,t} = \sum_{i=2～5} Q_{T,H,d,t,i} 
$$

$$
\large
Q_{UT,H,OR,d,t} = \sum_{i=2～5} Q_{UT,H,d,t,i} 
$$

$Q_{T,H,OR,d,t}$：居室のみを暖房する方式における日付$d$の時刻$t$における1時間当たりのその他の居室に設置された暖房設備の処理暖房負荷(MJ/h)  
$Q_{UT,H,OR,d,t}$：居室のみを暖房する方式における日付$d$の時刻$t$における1時間当たりのその他の居室に設置された暖房設備の未処理暖房負荷(MJ/h)  

# 要確認：(4a)がコメントアウトしている？

In [ ]:
# # その他の居室に設置された暖房設備の処理暖房負荷 (4a)
# def get_Q_T_H_OR_d_t():
#     return np.sum([get_Q_T_H_d_t_i(i) for i in range(2, 6)], axis=0)

# その他の居室に設置された暖房設備の未処理暖房負荷 (4b)
def get_Q_UT_H_OR_d_t(region, A_A, A_MR, A_OR, spec_MR, spec_OR, spec_HS, mode_MR, mode_OR, CG, L_T_H_d_t):
    # その他の居室がない場合
    if A_OR == 0:
        return np.zeros(24 * 365)
    else:
        if spec_OR['type'] in ['温水暖房用パネルラジエーター', '温水暖房用床暖房', '温水暖房用ファンコンベクター']:
            # 送水温度の決定
            Theta_SW_hs_op = hwh.get_Theta_SW_hs_op(spec_HS['type'], CG)
            rad_list = hwh.get_rad_list(spec_MR, spec_OR)
            p_hs_d_t = hwh.get_p_hs_d_t(Theta_SW_hs_op, rad_list, L_T_H_d_t, A_A, A_MR, A_OR, region, mode_MR,
                                        mode_OR)
            Theta_SW_d_t = hwh.get_Theta_SW_d_t(Theta_SW_hs_op, p_hs_d_t)

            # 未処理負荷
            Q_UT_H_d_t_i = np.zeros((5, 24 * 365))
            for i in range(2, 6):

                if spec_OR['type'] == '温水暖房用パネルラジエーター':
                    # 床面積
                    A_HCZ = get_A_HCZ_i(i, A_A, A_MR, A_OR)
                    R_type = 'その他の居室'

                    # パネルラジエーターの最大能力
                    q_max_rad = rad_panel.get_q_max_rad(region, mode_OR, A_HCZ, R_type)
                    # パネルラジエーターの最大暖房出力
                    Q_max_H_rad = rad_panel.get_Q_max_H_rad(Theta_SW_d_t, q_max_rad)
                elif spec_OR['type'] == '温水暖房用床暖房':
                    # 床面積
                    A_HCZ = get_A_HCZ_i(i, A_A, A_MR, A_OR)
                    r_Af = spec_OR['r_Af']
                    A_f = rad_floor.get_A_f(A_HCZ, r_Af)
                    # 温水床暖房の単位面積当たりの上面最大放熱能力
                    Q_max_H_rad = rad_floor.get_Q_max_H_rad(Theta_SW_d_t, A_f)
                elif spec_OR['type'] == '温水暖房用ファンコンベクター':
                    # 床面積
                    A_HCZ = get_A_HCZ_i(i, A_A, A_MR, A_OR)
                    R_type = 'その他の居室'
                    q_max_FC = rad_fanc.get_q_max_FC(region, mode_OR, A_HCZ, R_type)
                    Q_max_H_rad = rad_fanc.get_Q_max_H_rad(Theta_SW_d_t, q_max_FC)
                else:
                    raise ValueError(spec_OR['type'])

                # 処理負荷
                Q_T_H_d_t_i = np.min([Q_max_H_rad, L_T_H_d_t[i - 1]], axis=0)

                # 未処理負荷
                Q_UT_H_d_t_i[i - 1] = L_T_H_d_t[i - 1] - Q_T_H_d_t_i

            return np.sum(Q_UT_H_d_t_i, axis=0)
        else:
            return np.sum(
                [calc_Q_UT_H_d_t(i, spec_OR, A_A, A_MR, A_OR, region, mode_OR, L_T_H_d_t[i - 1]) for i in
                 range(2, 6)], axis=0)


def calc_Q_UT_H_d_t(i, device, A_A, A_MR, A_OR, region, mode, L_H_d_t):
    # 床面積
    A_HCZ = get_A_HCZ_i(i, A_A, A_MR, A_OR)

    if device['type'] == 'ルームエアコンディショナー':

        # 仕様の取得
        q_rtd_C = rac_spec.get_q_rtd_C(A_HCZ)
        q_rtd_H = rac_spec.get_q_rtd_H(q_rtd_C)
        e_rtd_C = rac_spec.get_e_rtd_C(device['e_class'], q_rtd_C)
        e_rtd_H = rac_spec.get_e_rtd_H(e_rtd_C)

        # 未処理負荷の計算
        Q_UT_H_d_t = rac.get_Q_UT_H_d_t(
            region=region,
            q_rtd_C=q_rtd_C,
            q_rtd_H=q_rtd_H,
            e_rtd_H=e_rtd_H,
            L_H_d_t=L_H_d_t,
        )

    elif device['type'] == 'FF暖房機':

        # 仕様の取得
        q_max_H = ff_spec.get_q_max_H(A_HCZ)
        q_min_H = ff_spec.get_q_min_H(q_max_H)
        P_rtd_H = ff_spec.get_P_rtd_H(q_max_H)
        P_itm_H = ff_spec.get_P_itm_H()

        # 未処理負荷の計算
        Q_UT_H_d_t = ff.get_Q_UT_H_d_t(
            q_max_H=q_max_H,
            L_H_d_t=L_H_d_t
        )

    elif device['type'] == '電気蓄熱暖房器':

        # 仕様の取得
        q_rq_H = ets_spec.get_q_rq_H(region)
        f_cT = ets_spec.get_f_cT(region)
        f_cI = ets_spec.get_f_cI(mode, '主たる居室' if i == 1 else 'その他の居室')
        q_rtd_H = ets_spec.get_q_rtd_H(q_rq_H, A_HCZ, f_cT, f_cI)
        e_rtd_H = ets_spec.get_e_rtd_H()

        # 未処理負荷の計算
        Q_UT_H_d_t = ets.get_Q_UT_H_d_t(
            q_rtd_H=q_rtd_H,
            e_rtd_H=e_rtd_H,
            L_H_d_t=L_H_d_t
        )

    elif device['type'] == '電気ヒーター床暖房':

        # 仕様の取得
        r_Af = device['r_Af']
        r_up = device['r_up']
        A_f = eheater_spec.get_A_f(A_HCZ, r_Af)

        # 未処理負荷の計算
        Q_UT_H_d_t = eheater.get_Q_UT_H_d_t(
            A_f=A_f,
            r_up=r_up,
            L_H_d_t=L_H_d_t
        )

    elif device['type'] in ['温水暖房用パネルラジエーター', '温水暖房用床暖房', '温水暖房用ファンコンベクター']:
        raise ValueError(device['type'])

    elif device['type'] == 'ルームエアコンディショナー付温水床暖房機':

        # 仕様の取得
        r_Af = device['r_Af']

        # 未処理負荷の計算
        Q_UT_H_d_t = racfh.get_Q_UT_H_d_t(region, A_HCZ, r_Af, L_H_d_t)

    else:
        raise ValueError(device['type'])

    return Q_UT_H_d_t

$$
\large
Q_{T,H,d,t,i} = min(Q_{max,H,d,t,i}, L_{H,d,t,i})
$$

$$
\large
Q_{UT,H,d,t,i} = L_{H,d,t,i} - Q_{T,H,d,t,i}
$$

$Q_{max,H,d,t,i}$：日付$d$の時刻$t$における1時間当たりの暖冷房区画$i$に設置された暖房設備機器等の最大暖房出力(MJ/h)  
$L_{H,d,t,i}$：日付$d$の時刻$t$における暖冷房区画$i$の1時間当たりの暖房負荷(MJ/h)  

$L_{H,d,t,i}$が0未満の場合は0とする。 

In [ ]:
# 暖冷房区画iに設置された暖房設備機器等の処理暖房負荷 (5a)
def get_Q_T_H_d_t_i(Q_max_H_d_t_i, L_H_d_t_i):
    assert len(Q_max_H_d_t_i.shape) == 1
    assert len(L_H_d_t_i.shape) == 1
    return np.min([Q_max_H_d_t_i, L_H_d_t_i], axis=0)

# 暖冷房区画iに設置された暖房設備機器等の未処理暖房負荷 (5b)
def get_Q_UT_H_d_t_i(Q_T_H_d_t_i, L_H_d_t_i):
    # 1 時間当たりの暖房負荷
    __L_H_d_t_i = np.max([np.zeros(24 * 365), L_H_d_t_i], axis=0)  # 0未満の場合は0

    return __L_H_d_t_i - Q_T_H_d_t_i

#### 6.2 暖房設備のエネルギー消費量

$$
\large
E_{E,H,d,t} = E_{E,H,hs,d,t} + E_{E,H,aux,ass,d,t}
$$

$$
\large
E_{G,H,d,t} = E_{G,H,hs,d,t}
$$

$$
\large
E_{K,H,d,t} = E_{K,H,hs,d,t}
$$

$$
\large
E_{M,H,d,t} = E_{M,H,hs,d,t}
$$

$E_{E,H,d,t}$：日付$d$の時刻$t$における1時間当たりの暖房設備の消費電力量(kWh/h)  
$E_{G,H,d,t}$：日付$d$の時刻$t$における1時間当たりの暖房設備のガス消費量(MJ/h)  
$E_{K,H,d,t}$：日付$d$の時刻$t$における1時間当たりの暖房設備の灯油消費量(MJ/h)  
$E_{M,H,d,t}$：日付$d$の時刻$t$における1時間当たりの暖房設備のその他の燃料による一次エネルギー消費量(MJ/h)  
$E_{E,H,hs,d,t}$：日付$d$の時刻$t$における1時間当たりの暖房設備機器等の消費電力量(kWh/h)  
$E_{G,H,hs,d,t}$：日付$d$の時刻$t$における1時間当たりの暖房設備機器等のガス消費量(MJ/h)  
$E_{K,H,hs,d,t}$：日付$d$の時刻$t$における1時間当たりの暖房設備機器等の灯油消費量(MJ/h)  
$E_{M,H,hs,d,t}$：日付$d$の時刻$t$における1時間当たりの暖房設備機器等のその他の燃料による一次エネルギー消費量(MJ/h)  
$E_{E,H,aux,ass,d,t}$：日付$d$の時刻$t$における1時間当たりの空気集熱式太陽熱利用設備の補機の消費電力量のうちの暖房設備への付加分(kWh/h)  

日付$d$の時刻$t$における1 時間当たりの空気集熱式太陽熱利用設備の補機の消費電力量のうちの暖房設備への付加分$E_{E,H,aux,ass,d,t}$は、空気集熱式太陽熱利用設備を採用しない場合はゼロとする。

In [ ]:
# 暖房設備の消費電力量（kWh/h）(6a)
def get_E_E_H_d_t(region, sol_region, A_A, A_MR, A_OR,
                  H_A=None,
                  spec_MR=None,
                  spec_OR=None, spec_HS=None,
                  mode_MR=None, mode_OR=None,
                  CG=None,
                  SHC=None, heating_flag_d=None,
                  L_H_A_d_t=None, L_T_H_d_t_i=None,
                  **args):

    if region == 8:
        return np.zeros(24 * 365)

    # 暖房設備機器等の消費電力量
    E_E_hs_d_t = get_E_E_hs_d_t(region, A_A, A_MR, A_OR, H_A, spec_MR, spec_OR, spec_HS, mode_MR, mode_OR, CG,
                                L_H_A_d_t, L_T_H_d_t_i)

    # 空気集熱式太陽熱利用設備の補機の消費電力量のうちの暖房設備への付加分
    E_E_aux_ass_d_t = get_E_E_aux_ass_d_t(SHC, heating_flag_d, region, sol_region)

    return E_E_hs_d_t + E_E_aux_ass_d_t


# 空気集熱式太陽熱利用設備の補機の消費電力量のうちの暖房設備への付加分
def get_E_E_aux_ass_d_t(SHC, heating_flag_d, region, sol_region):
    if SHC is not None and SHC['type'] == '空気集熱式':
        E_E_aux_ass_d_t = ass.calc_E_E_H_aux_ass_d_t(
            hotwater_use=SHC['hotwater_use'],
            heating_flag_d=heating_flag_d,
            region=region,
            sol_region=sol_region,
            P_alpha=SHC['P_alpha'],
            P_beta=SHC['P_beta'],
            A_col=SHC['A_col'],
            V_fan_P0=SHC['V_fan_P0'],
            m_fan_test=SHC['m_fan_test'],
            d0=SHC['d0'],
            d1=SHC['d1'],
            fan_sso=SHC['fan_sso'],
            fan_type=SHC['fan_type'],
        )
    else:
        E_E_aux_ass_d_t = np.zeros(24 * 365)

    print('空気集熱式太陽熱利用設備の補機の消費電力量のうちの暖房設備への付加分 E_E_aux_ass = {} [kWh/年]'.format(np.sum(E_E_aux_ass_d_t)))

    return E_E_aux_ass_d_t


# 暖房設備機器等の消費電力量（kWh/h）(7a,8a,9a)
def get_E_E_hs_d_t(region, A_A, A_MR, A_OR, H_A, spec_MR, spec_OR, spec_HS, mode_MR, mode_OR, CG, L_H_A_d_t,
                   L_T_H_d_t_i):
    if H_A is not None:
        return get_E_E_H_hs_A_d_t(region, A_A, H_A, L_H_A_d_t)
    elif (spec_MR is not None or spec_OR is not None) and L_T_H_d_t_i is not None:
        if is_hotwaterheatingonly(spec_MR, spec_OR):
            # 居室のみを暖房する方式でかつ主たる居室とその他の居室ともに温水暖房を設置する場合 (8a)
            return get_E_E_H_hs_MROR_d_t(region, A_A, A_MR, A_OR, spec_MR, spec_OR, spec_HS, CG, L_T_H_d_t_i)
        else:
            # 居室のみを暖房する方式でかつ主たる居室とその他の居室ともに温水暖房を設置する場合に該当しない場合 (9a)
            return get_E_E_H_hs_MR_d_t(region, A_A, A_MR, A_OR, mode_MR, mode_OR, spec_MR, spec_OR, spec_HS,
                                       L_T_H_d_t_i, CG) \
                   + get_E_E_H_hs_OR_d_t(region, A_A, A_MR, A_OR, mode_MR, mode_OR, spec_MR, spec_OR, spec_HS,
                                         L_T_H_d_t_i, CG)
    else:
        return np.zeros(24*365)


def is_hotwaterheatingonly(H_MR, H_OR):
    from section4_7 import get_rad_type_list
    rad_types = get_rad_type_list()

    if H_MR['type'] not in rad_types:
        return False

    if H_OR is None:
        return True

    if H_OR['type'] in rad_types:
        return True

    return False

def get_virtual_heatsource(region, H_HS):
    default_HS = get_default_heatsource(region)

    if H_HS is None:
        return default_HS
    elif H_HS['type'] == 'その他の温水暖房機':
        return {
            'type': default_HS['type'],
            'e_rtd_hs': default_HS['e_rtd_hs'],
            'pipe_insulation': H_HS['pipe_insulation'],
            'underfloor_pipe_insulation': H_HS['underfloor_pipe_insulation'],
        }
    elif H_HS['type'] == '温水暖房機を設置しない':
        return {
            'type': default_HS['type'],
            'e_rtd_hs': default_HS['e_rtd_hs'],
            'pipe_insulation': False,
            'underfloor_pipe_insulation': False,
        }
    else:
        return H_HS


# 暖房区画1つと1つの暖房設備機器によって消費される電力消費量
def calc_E_E_H_d_t(i, device, A_A, A_MR, A_OR, region, mode, L_H_d_t):
    # 床面積の取得
    A_HCZ = get_A_HCZ_i(i, A_A, A_MR, A_OR)

    if device['type'] == 'ルームエアコンディショナー':

        # 仕様の取得
        q_rtd_C = rac_spec.get_q_rtd_C(A_HCZ)
        q_rtd_H = rac_spec.get_q_rtd_H(q_rtd_C)
        e_rtd_C = rac_spec.get_e_rtd_C(device['e_class'], q_rtd_C)
        e_rtd_H = rac_spec.get_e_rtd_H(e_rtd_C)

        # 消費電力量の計算
        E_E_H_d_t = rac.get_E_E_H_d_t(
            region=region,
            q_rtd_C=q_rtd_C,
            q_rtd_H=q_rtd_H,
            e_rtd_H=e_rtd_H,
            dualcompressor=device['dualcompressor'],
            L_H_d_t=L_H_d_t,
        )

    elif device['type'] == 'FF暖房機':

        # 仕様の取得
        q_max_H = ff_spec.get_q_max_H(A_HCZ)
        q_min_H = ff_spec.get_q_min_H(q_max_H)
        P_rtd_H = ff_spec.get_P_rtd_H(q_max_H)
        P_itm_H = ff_spec.get_P_itm_H()

        # 消費電力量の計算
        E_E_H_d_t = ff.get_E_E_H_d_t(
            q_max_H=q_max_H,
            q_min_H=q_min_H,
            P_rtd_H=P_rtd_H,
            P_itm_H=P_itm_H,
            L_H_d_t=L_H_d_t
        )

    elif device['type'] == '電気ヒーター床暖房':

        # 仕様の取得
        r_Af = device['r_Af']
        r_up = device['r_up']
        A_f = eheater_spec.get_A_f(A_HCZ, r_Af)

        # 消費電力量の計算
        E_E_H_d_t = eheater.get_E_E_H_d_t(
            A_f=A_f,
            r_up=r_up,
            L_H_d_t=L_H_d_t
        )

    elif device['type'] == '電気蓄熱暖房器':

        # 仕様の取得
        q_rq_H = ets_spec.get_q_rq_H(region)
        f_cT = ets_spec.get_f_cT(region)
        f_cI = ets_spec.get_f_cI(mode, '主たる居室' if i == 1 else 'その他の居室')
        q_rtd_H = ets_spec.get_q_rtd_H(q_rq_H, A_HCZ, f_cT, f_cI)
        e_rtd_H = ets_spec.get_e_rtd_H()

        # 消費電力量の計算
        E_E_H_d_t = ets.get_E_E_H_d_t(
            q_rtd_H=q_rtd_H,
            e_rtd_H=e_rtd_H,
            L_H_d_t=L_H_d_t
        )

    elif device['type'] in ['温水暖房用パネルラジエーター', '温水暖房用床暖房', '温水暖房用ファンコンベクター']:
        # 温水暖房は第7節で取り扱うためここでは処理しない
        raise UserWarning(device['type'])

    elif device['type'] == 'ルームエアコンディショナー付温水床暖房機':

        # 仕様の取得
        r_Af = device['r_Af']
        r_up = device['r_up']
        pipe_insulation = device['pipe_insulation']

        # 消費電力量の計算
        E_E_H_d_t = racfh.get_E_E_d_t(
            region=region,
            A_A_act=A_A,
            i=i,
            A_HCZ=A_HCZ,
            r_Af=r_Af,
            r_up=r_up,
            pipe_insulation=pipe_insulation,
            L_H_d_t=L_H_d_t
        )

    else:
        raise ValueError(device['type'])

    print('{} E_E_H_d_t_{} = {} [kWh] (L_H_d_t_{} = {} [MJ])'.format(device['type'], i, np.sum(E_E_H_d_t), i,
                                                                     np.sum(L_H_d_t)))

    return E_E_H_d_t


def get_L_H_d_t(region, sol_region, A_A, A_MR, A_OR, mode_H, H_MR, H_OR, mode_MR, mode_OR, Q, mu_H, TS, r_A_ufvnt, HEX,
                SHC, underfloor_insulation, normalize=True):
    # 温水暖房
    floorheating_i = get_floorheating_i(H_MR, H_OR)
    R_l_i = get_R_l_i(H_MR, H_OR)

    # 運転モード
    if mode_H == '住戸全体を連続的に暖房する方式':
        # 全館連続
        mode = tuple(["全館連続"] * 12)
    else:
        mode = (mode_MR, None, mode_OR, mode_OR, mode_OR, None, None, None, None, None, None, None)

    # 暖冷房区画i݅の1時間当たりの暖房負荷 (1)
    args = {
        'region': region,
        'A_A': A_A,
        'A_MR': A_MR,
        'A_OR': A_OR,
        'mode': mode,

        # 外皮
        'Q': Q,
        'mu_H': mu_H,
        'TS': TS,

        # 床下換気
        'r_A_ufvnt': r_A_ufvnt,
        'underfloor_insulation': underfloor_insulation,

        # 床下暖房
        'floorheating': tuple(floorheating_i),
        'R_l_i': tuple(R_l_i),
    }

    # 熱交換型換気
    if HEX is not None:
        args['hex'] = True
        args['etr_dash_t'] = calc_etr_dash_t(
            etr_t_raw=HEX['etr_t'],
            e=HEX['e'],
            C_bal=HEX['C_bal'],
            C_leak=HEX['C_leak']
        )
    else:
        args['hex'] = False
        args['etr_dash_t'] = None

    # 太陽熱集熱式
    if SHC is not None and SHC['type'] == '空気集熱式':
        args.update({
            'hotwater_use': SHC['hotwater_use'],
            'supply_target': SHC['supply_target'],
            'sol_region': sol_region,
            'P_alpha': SHC['P_alpha'],
            'P_beta': SHC['P_beta'],
            'A_col': SHC['A_col'],
            'V_fan_P0': SHC['V_fan_P0'],
            'm_fan_test': SHC['m_fan_test'],
            'd0': SHC['d0'],
            'd1': SHC['d1'],
            'ufv_insulation': SHC['ufv_insulation'],
            'r_A_ufvnt_ass': SHC['r_A_ufvnt_ass'],
        })

    L_T_H_d_t_i, L_dash_H_R_d_t_i = ld.get_L_H_d_t_i(**args)

    if normalize:
        L_T_H_d_t_i[L_T_H_d_t_i < 0] = 0

    return L_T_H_d_t_i, L_dash_H_R_d_t_i


def get_floorheating_i(H_MR, H_OR):
    floorheating = np.zeros(12)

    if H_MR is not None:
        if H_MR['type'] in ['電気ヒーター床暖房', '温水暖房用床暖房', 'ルームエアコンディショナー付温水床暖房機']:
            floorheating[0] = True

    if H_OR is not None:
        if H_OR['type'] in ['電気ヒーター床暖房', '温水暖房用床暖房', 'ルームエアコンディショナー付温水床暖房機']:
            floorheating[1:6] = True

    return floorheating


def get_R_l_i(H_MR, H_OR):
    # 床暖房
    R_l_i = np.zeros(12)

    if H_MR is not None:
        if H_MR['type'] in ['電気ヒーター床暖房', '温水暖房用床暖房', 'ルームエアコンディショナー付温水床暖房機']:
            if 'r_dash_Af' not in H_MR:
                R_l_i[0] = H_MR['r_Af']
            else:
                # 吹き抜けを有する場合
                R_l_i[0] = H_MR['r_dash_Af']

    if H_OR is not None:
        if H_OR['type'] in ['電気ヒーター床暖房', '温水暖房用床暖房', 'ルームエアコンディショナー付温水床暖房機']:
            R_l_i[1:6] = H_OR['r_Af']

    return R_l_i


def get_A_HCZ_i(i, A_A, A_MR, A_OR):
    return ld.get_A_HCZ_i(i, A_A, A_MR, A_OR)


# 暖房設備のガス消費量（MJ/h）(6b)
def get_E_G_H_d_t(region, A_A, A_MR, A_OR, H_A, spec_MR, spec_OR, spec_HS, mode_MR, mode_OR, CG, L_T_H_d_t_i):
    return get_E_G_hs_d_t(region, A_A, A_MR, A_OR, H_A, spec_MR, spec_OR, spec_HS, mode_MR, mode_OR, L_T_H_d_t_i, CG)


# 暖房設備機器等のガス消費量（MJ/h）(7b,8b,9b)
def get_E_G_hs_d_t(region, A_A, A_MR, A_OR, H_A, spec_MR, spec_OR, spec_HS, mode_MR, mode_OR, L_T_H_d_t_i, CG):
    if H_A is not None:
        # (7b)
        return get_E_G_H_hs_A_d_t()
    elif (spec_MR is not None or spec_OR is not None) and L_T_H_d_t_i is not None:
        if is_hotwaterheatingonly(spec_MR, spec_OR):
            # 居室のみを暖房する方式でかつ主たる居室とその他の居室ともに温水暖房を設置する場合 (8b)
            return get_E_G_H_hs_MROR_d_t(mode_MR, mode_OR, spec_HS, spec_MR, spec_OR, L_T_H_d_t_i, CG, A_A, A_MR, A_OR,
                                         region)
        else:
            # 居室のみを暖房する方式でかつ主たる居室とその他の居室ともに温水暖房を設置する場合に該当しない場合
            # (9b)
            return get_E_G_H_hs_MR_d_t(region, A_A, A_MR, A_OR, mode_MR, mode_OR, spec_MR, spec_OR, spec_HS,
                                       L_T_H_d_t_i, CG) \
                   + get_E_G_H_hs_OR_d_t(region, A_A, A_MR, A_OR, mode_MR, mode_OR, spec_MR, spec_OR, spec_HS,
                                         L_T_H_d_t_i, CG)
    else:
        return np.zeros(24*365)


# 暖房区画1つと1つの暖房設備機器によって消費されるガス消費量
def calc_E_G_H_d_t(i, device, A_A, A_MR, A_OR, L_H_d_t):
    # 床面積
    A_HCZ = get_A_HCZ_i(i, A_A, A_MR, A_OR)

    if device['type'] == 'ルームエアコンディショナー':
        # ガス消費量の計算
        E_G_H_d_t = rac.get_E_G_H_d_t()

    elif device['type'] == 'FF暖房機':

        # 仕様の取得
        q_max_H = ff_spec.get_q_max_H(A_HCZ)
        if 'e_rtd_H' in device:
            e_rtd_H = device['e_rtd_H']
        else:
            e_rtd_H = ff_spec.get_e_rtd_H_default()

        # ガス消費量の計算
        E_G_H_d_t = ff.get_E_G_H_d_t(
            fuel='K',
            q_max_H=q_max_H,
            e_rtd_H=e_rtd_H,
            L_H_d_t=L_H_d_t
        )

    elif device['type'] == '電気蓄熱暖房器':
        # ガス消費量の計算
        E_G_H_d_t = ets.get_E_G_H_d_t()

    elif device['type'] == '電気ヒーター床暖房':
        # ガス消費量の計算
        E_G_H_d_t = eheater.get_E_G_H_d_t()

    elif device['type'] in ['温水暖房用パネルラジエーター', '温水暖房用床暖房', '温水暖房用ファンコンベクター']:
        # 温水暖房は第7節で取り扱うためここでは処理しない
        raise UserWarning(device['type'])

    elif device['type'] == 'ルームエアコンディショナー付温水床暖房機':
        # ガス消費量の計算
        E_G_H_d_t = racfh.get_E_G_d_t()

    else:
        raise ValueError(device['type'])

    print('{} E_G_H_d_t_{} = {} [MJ] (L_H_d_t_{} = {} [MJ])'.format(device['type'], i, np.sum(E_G_H_d_t), i,
                                                                    np.sum(L_H_d_t)))

    return E_G_H_d_t


# 暖房設備の灯油消費量（MJ/h）(6c)
def get_E_K_H_d_t(region, A_A, A_MR, A_OR, H_A, spec_MR, spec_OR, spec_HS, mode_MR, mode_OR, CG, L_T_H_d_t_i):
    return get_E_K_hs_d_t(region, A_A, A_MR, A_OR, H_A, spec_MR, spec_OR, spec_HS, mode_MR, mode_OR, L_T_H_d_t_i,
                          CG)


# 暖房設備機器等の灯油消費量（MJ/h）(7c,8c,9c)
def get_E_K_hs_d_t(region, A_A, A_MR, A_OR, H_A, spec_MR, spec_OR, spec_HS, mode_MR, mode_OR, L_T_H_d_t_i, CG):
    if H_A is not None:
        # (7c)
        return get_E_K_H_hs_A_d_t()
    elif (spec_MR is not None or spec_OR is not None) and L_T_H_d_t_i is not None:
        if is_hotwaterheatingonly(spec_MR, spec_OR):
            # 居室のみを暖房する方式でかつ主たる居室とその他の居室ともに温水暖房を設置する場合 (8c)
            return get_E_K_H_hs_MROR_d_t(spec_HS, spec_MR, spec_OR, CG, A_A, A_MR, A_OR, region, mode_MR, mode_OR,
                                         L_T_H_d_t_i)
        else:

            # 居室のみを暖房する方式でかつ主たる居室とその他の居室ともに温水暖房を設置する場合に該当しない場合 (9c)
            return get_E_K_H_hs_MR_d_t(region, A_A, A_MR, A_OR, spec_MR, spec_OR, spec_HS, mode_MR, mode_OR,
                                       L_T_H_d_t_i, CG) \
                   + get_E_K_H_hs_OR_d_t(region, A_A, A_MR, A_OR, spec_MR, spec_OR, spec_HS, mode_MR, mode_OR,
                                         L_T_H_d_t_i, CG)
    else:
        return np.zeros(24*365)

# 実質的な暖房機器の仕様を取得
def get_virtual_heating_devices(region, H_MR, H_OR):
    # 規定の暖房機器を取得
    default_spec = get_default_heating_spec(region)

    # 設置しないまたはその他の機器の場合は規定の機器に置き換える
    if H_MR is not None:
        if H_MR['type'] == '設置しない' or H_MR['type'] == 'その他':
            H_MR = default_spec[0]
    if H_OR is not None:
        if H_OR['type'] == '設置しない' or H_OR['type'] == 'その他':
            H_OR = default_spec[1]

    return H_MR, H_OR


# 暖房区画1つと1つの暖房設備機器によって消費されるガス消費量
def calc_E_K_d_t(i, device, A_A, A_MR, A_OR, L_H_d_t):
    # 床面積
    A_HCZ = get_A_HCZ_i(i, A_A, A_MR, A_OR)

    if device['type'] == 'ルームエアコンディショナー':
        # 灯油の消費量の計算
        E_K_d_t = rac.get_E_K_H_d_t()

    elif device['type'] == 'FF暖房機':

        # 仕様の取得
        q_max_H = ff_spec.get_q_max_H(A_HCZ)
        if 'e_rtd_H' in device:
            e_rtd_H = device['e_rtd_H']
        else:
            e_rtd_H = ff_spec.get_e_rtd_H_default()

        # ガス消費量の計算
        E_K_d_t = ff.get_E_K_H_d_t(
            fuel='K',
            q_max_H=q_max_H,
            e_rtd_H=e_rtd_H,
            L_H_d_t=L_H_d_t
        )

    elif device['type'] == '電気蓄熱暖房器':
        # 灯油の消費量の計算
        E_K_d_t = ets.get_E_K_H_d_t()

    elif device['type'] == '電気ヒーター床暖房':
        # 灯油の消費量の計算
        E_K_d_t = eheater.get_E_K_H_d_t()

    elif device['type'] in ['温水暖房用パネルラジエーター', '温水暖房用床暖房', '温水暖房用ファンコンベクター']:
        # 温水暖房は第7節で取り扱うためここでは処理しない
        raise UserWarning(device['type'])

    elif device['type'] == 'ルームエアコンディショナー付温水床暖房機':
        # 灯油の消費量の計算
        E_K_d_t = racfh.get_E_K_d_t()

    else:
        raise ValueError(device['type'])

    print(
        '{} E_K_d_t_{} = {} [MJ] (L_H_d_t_{} = {} [MJ)'.format(device['type'], i, np.sum(E_K_d_t), i, np.sum(L_H_d_t)))

    return E_K_d_t


# 暖房設備のその他の燃料による一次エネルギー消費量（MJ/h）(6d)
def get_E_M_H_d_t(region, A_A, A_MR, A_OR, H_A, spec_MR, spec_OR, spec_HS, L_T_H_d_t_i):
    return get_E_M_hs_d_t(region, A_A, A_MR, A_OR, H_A, spec_MR, spec_OR, spec_HS, L_T_H_d_t_i)


# 暖房設備機器等のその他の燃料による一次エネルギー消費量（MJ/h）(7d,8d,9d)
def get_E_M_hs_d_t(region, A_A, A_MR, A_OR, H_A, spec_MR, spec_OR, spec_HS, L_H_d_t):
    if H_A is not None:
        # (7d)
        return get_E_M_H_hs_A_d_t()
    else:

        if is_hotwaterheatingonly(spec_MR, spec_OR):
            # 居室のみを暖房する方式でかつ主たる居室とその他の居室ともに温水暖房を設置する場合
            # (8d)
            return get_E_M_H_hs_MROR_d_t(region, spec_HS)
        else:
            # 居室のみを暖房する方式でかつ主たる居室とその他の居室ともに温水暖房を設置する場合に該当しない場合
            # (9d)
            return get_E_M_H_hs_MR_d_t(A_A, A_MR, A_OR, spec_MR, spec_HS, L_H_d_t) + \
                   get_E_M_H_hs_OR_d_t(A_A, A_MR, A_OR, spec_OR, spec_HS, L_H_d_t)


#### 6.3 暖房設備機器のエネルギー消費量 

#### 6.3.1 住戸全体を連続的に暖房する方式

$$
\large
E_{E,H,hs,d,t} = E_{E,H,hs,A,d,t}
$$

$$
\large
E_{G,H,hs,d,t} = E_{G,H,hs,A,d,t}
$$

$$
\large
E_{K,H,hs,d,t} = E_{K,H,hs,A,d,t}
$$

$$
\large
E_{M,H,hs,d,t} = E_{M,H,hs,A,d,t}
$$

$E_{E,H,hs,d,t}$：日付$d$の時刻$t$における1時間当たりの暖房設備機器の消費電力量(kWh/h)  
$E_{G,H,hs,d,t}$：日付$d$の時刻$t$における1時間当たりの暖房設備機器のガス消費量(MJ/h)  
$E_{K,H,hs,d,t}$：日付$d$の時刻$t$における1時間当たりの暖房設備機器の灯油消費量(MJ/h)  
$E_{M,H,hs,d,t}$：日付$d$の時刻$t$における1時間当たりの暖房設備機器のその他の燃料による一次エネルギー消費量(MJ/h)  
$E_{E,H,hs,A,d,t}$：住戸全体を連続的に暖房する方式における日付$d$の時刻$t$における1時間当たりの暖房設備機器の消費電力量(kWh/h)  
$E_{G,H,hs,A,d,t}$：住戸全体を連続的に暖房する方式における日付$d$の時刻$t$における1時間当たりの暖房設備機器のガス消費量(MJ/h)  
$E_{K,H,hs,A,d,t}$：住戸全体を連続的に暖房する方式における日付$d$の時刻$t$における1時間当たりの暖房設備機器の灯油消費量(MJ/h)  
$E_{M,H,hs,A,d,t}$：住戸全体を連続的に暖房する方式における日付$d$の時刻$t$における1時間当たりの暖房設備機器のその他の燃料による一次エネルギー消費量(MJ/h)  

In [ ]:
# 1時間当たりの暖房設備機器の消費電力量 (7a)
def get_E_E_H_hs_A_d_t(region, A_A, H_A, L_H_A_d_t):
    # 仕様の取得
    if 'q_rtd_H' in H_A and 'P_rtd_H' in H_A:
        q_rtd_H = H_A['q_rtd_H']
        P_rtd_H = H_A['P_rtd_H']
    else:
        q_rtd_H = dc_spec.get_q_rtd_H(region, A_A)
        P_rtd_H = dc_spec.get_P_rtd_H(q_rtd_H)
    C_pl_H = H_A['C_pl_H']

    # 電力消費量の計算
    E_E_H_d_t = dc.get_E_E_H_d_t(
        region=region,
        q_rtd_H=q_rtd_H,
        P_rtd_H=P_rtd_H,
        C_pl_H=C_pl_H,
        L_H_d_t=L_H_A_d_t
    )

    return E_E_H_d_t


# 1時間当たりの暖房設備機器のガス電力量 (7b)
def get_E_G_H_hs_A_d_t():
    return dc.get_E_G_H_d_t()


# 1時間当たりの暖房設備機器の灯油電力量 (7c)
def get_E_K_H_hs_A_d_t(**args):
    return dc.get_E_K_H_d_t()


# 1時間当たりの暖房設備機器のその他の一次エネルギー消費量 (7d)
def get_E_M_H_hs_A_d_t():
    return dc.get_E_M_H_d_t()

#### 6.3.2 居室のみを暖房する方式でかつ主たる居室とその他の居室ともに温水暖房を設置する場合

$$
\large
E_{E,H,hs,d,t} = E_{E,H,hs,MROR,d,t}
$$

$$
\large
E_{G,H,hs,d,t} = E_{G,H,hs,MROR,d,t}
$$

$$
\large
E_{K,H,hs,d,t} = E_{K,H,hs,MROR,d,t}
$$

$$
\large
E_{M,H,hs,d,t} = E_{M,H,hs,MROR,d,t}
$$

$E_{E,H,hs,d,t}$：日付$d$の時刻$t$における1時間当たりの暖房設備機器の消費電力量(kWh/h)  
$E_{G,H,hs,d,t}$：日付$d$の時刻$t$における1時間当たりの暖房設備機器のガス消費量(MJ/h)  
$E_{K,H,hs,d,t}$：日付$d$の時刻$t$における1時間当たりの暖房設備機器の灯油消費量(MJ/h)  
$E_{M,H,hs,d,t}$：日付$d$の時刻$t$における1時間当たりの暖房設備機器のその他の燃料による一次エネルギー消費量(MJ/h)  
$E_{E,H,hs,MROR,d,t}$：居室のみを暖房する方式における日付$d$の時刻$t$における1時間当たりの主たる居室及びその他の居室に設置された暖房設備の消費電力量(kWh/h)  
$E_{G,H,hs,MROR,d,t}$：居室のみを暖房する方式における日付$d$の時刻$t$における1時間当たりの主たる居室及びその他の居室に設置された暖房設備のガス消費量(MJ/h)  
$E_{K,H,hs,MROR,d,t}$：居室のみを暖房する方式における日付$d$の時刻$t$における1時間当たりの主たる居室及びその他の居室に設置された暖房設備の灯油消費量(MJ/h)  
$E_{M,H,hs,MROR,d,t}$：居室のみを暖房する方式における日付$d$の時刻$t$における1時間当たりの主たる居室及びその他の居室に設置された暖房設備のその他の燃料による一次エネルギー消費量(MJ/h)  

In [ ]:
# 1時間当たりの暖房設備機器の消費電力量 (8a)
def get_E_E_H_hs_MROR_d_t(region, A_A, A_MR, A_OR, H_MR, H_OR, H_HS, CG, L_T_H_d_t_i):
    # 暖房方式及び運転方法の区分
    mode_MR, mode_OR = get_heating_mode(region=region, H_MR=H_MR, H_OR=H_OR)

    return hwh.get_E_E_H_d_t(H_HS, H_MR, H_OR, A_A, A_MR, A_OR, region, mode_MR, mode_OR, L_T_H_d_t_i, CG)


# 1時間当たりの暖房設備機器のガス電力量 (8b)
def get_E_G_H_hs_MROR_d_t(mode_MR, mode_OR, spec_HS, spec_MR, spec_OR, L_T_H_d_t_i, CG, A_A, A_MR, A_OR, region):
    return hwh.get_E_G_H_d_t(spec_HS, spec_MR, spec_OR, A_A, A_MR, A_OR, region, mode_MR, mode_OR, L_T_H_d_t_i, CG)


# 1時間当たりの暖房設備機器の灯油電力量 (8c)
def get_E_K_H_hs_MROR_d_t(spec_HS, spec_MR, spec_OR, CG, A_A, A_MR, A_OR, region, mode_MR, mode_OR, L_T_H_d_t_i):
    return hwh.get_E_K_H_d_t(spec_HS, spec_MR, spec_OR, A_A, A_MR, A_OR, region, mode_MR, mode_OR, L_T_H_d_t_i, CG)


# 1時間当たりの暖房設備機器のその他の一次エネルギー消費量 (8d)
def get_E_M_H_hs_MROR_d_t(region, H_HS):
    # 実質的な温水暖房熱源の取得
    spec_HS = get_virtual_heatsource(region, H_HS)

    return hwh.get_E_M_H_d_t(spec_HS)

#### 6.3.3 居室のみを暖房する方式でかつ主たる居室とその他の居室ともに温水暖房を設置する場合に該当しない場合 

$$
\large
E_{E,H,hs,d,t} = E_{E,H,hs,MR,d,t} + E_{E,H,hs,OR,d,t}
$$

$$
\large
E_{G,H,hs,d,t} = E_{G,H,hs,MR,d,t} + E_{G,H,hs,OR,d,t}
$$

$$
\large
E_{K,H,hs,d,t} = E_{K,H,hs,MR,d,t} + E_{K,H,hs,OR,d,t}
$$

$$
\large
E_{M,H,hs,d,t} = E_{M,H,hs,MR,d,t} + E_{M,H,hs,OR,d,t}
$$

$E_{E,H,hs,d,t}$：日付$d$の時刻$t$における1時間当たりの暖房設備機器の消費電力量(kWh/h)  
$E_{G,H,hs,d,t}$：日付$d$の時刻$t$における1時間当たりの暖房設備機器のガス消費量(MJ/h)  
$E_{K,H,hs,d,t}$：日付$d$の時刻$t$における1時間当たりの暖房設備機器の灯油消費量(MJ/h)  
$E_{M,H,hs,d,t}$：日付$d$の時刻$t$における1時間当たりの暖房設備機器のその他の燃料による一次エネルギー消費量(MJ/h)  
$E_{E,H,hs,MR,d,t}$：居室のみを暖房する方式における日付$d$の時刻$t$における1時間当たりの主たる居室に設置された暖房設備機器の消費電力量(kWh/h)  
$E_{G,H,hs,MR,d,t}$：居室のみを暖房する方式における日付$d$の時刻$t$における1時間当たりの主たる居室に設置された暖房設備機器のガス消費量(MJ/h)  
$E_{K,H,hs,MR,d,t}$：居室のみを暖房する方式における日付$d$の時刻$t$における1時間当たりの主たる居室に設置された暖房設備機器の灯油消費量(MJ/h)  
$E_{M,H,hs,MR,d,t}$：居室のみを暖房する方式における日付$d$の時刻$t$における1時間当たりの主たる居室に設置された暖房設備機器のその他の燃料による一次エネルギー消費量(MJ/h)  
$E_{E,H,hs,OR,d,t}$：居室のみを暖房する方式における日付$d$の時刻$t$における1時間当たりのその他の居室に設置された暖房設備機器の消費電力量(kWh/h)  
$E_{G,H,hs,OR,d,t}$：居室のみを暖房する方式における日付$d$の時刻$t$における1時間当たりのその他の居室に設置された暖房設備機器のガス消費量(MJ/h)  
$E_{K,H,hs,OR,d,t}$：居室のみを暖房する方式における日付$d$の時刻$t$における1時間当たりのその他の居室に設置された暖房設備機器の灯油消費量(MJ/h)  
$E_{M,H,hs,OR,d,t}$：居室のみを暖房する方式における日付$d$の時刻$t$における1時間当たりのその他の居室に設置された暖房設備機器のその他の燃料による一次エネルギー消費量(MJ/h)  

In [ ]:
# (9a)
def get_E_E_H_hs_MR_d_t(region, A_A, A_MR, A_OR, mode_MR, mode_OR, spec_MR, spec_OR, spec_HS, L_T_H_d_t_i, CG=None):
    if spec_MR['type'] in ['温水暖房用パネルラジエーター', '温水暖房用床暖房', '温水暖房用ファンコンベクター']:
        return hwh.get_E_E_H_d_t(
            H_HS=spec_HS,
            H_MR=spec_MR,
            H_OR=spec_OR,
            A_A=A_A,
            A_MR=A_MR,
            A_OR=A_OR,
            region=region,
            mode_MR=mode_MR,
            mode_OR=mode_OR,
            L_T_H_rad=L_T_H_d_t_i,
            CG=CG)
    else:
        return calc_E_E_H_d_t(1, spec_MR, A_A, A_MR, A_OR, region, mode_MR, L_T_H_d_t_i[0])
    
# (9b)
def get_E_G_H_hs_MR_d_t(region, A_A, A_MR, A_OR, mode_MR, mode_OR, spec_MR, spec_OR, spec_HS, L_T_H_d_t, CG):
    if spec_MR['type'] in ['温水暖房用パネルラジエーター', '温水暖房用床暖房', '温水暖房用ファンコンベクター']:
        return hwh.get_E_G_H_d_t(
            H_HS=spec_HS,
            H_MR=spec_MR,
            H_OR=spec_OR,
            A_A=A_A,
            A_MR=A_MR,
            A_OR=A_OR,
            region=region,
            mode_MR=mode_MR,
            mode_OR=mode_OR,
            L_T_H_rad=L_T_H_d_t,
            CG=CG)
    else:

        return calc_E_G_H_d_t(1, spec_MR, A_A, A_MR, A_OR, L_T_H_d_t[0])
    
# 居室のみを暖房する方式における日付の時刻における 1 時間当たりの主たる居室に設置された暖房設備機器のガス消費量 (9c)
def get_E_K_H_hs_MR_d_t(region, A_A, A_MR, A_OR, spec_MR, spec_OR, spec_HS, mode_MR, mode_OR, L_T_H_d_t, CG):
    if spec_MR['type'] in ['温水暖房用パネルラジエーター', '温水暖房用床暖房', '温水暖房用ファンコンベクター']:
        return hwh.get_E_K_H_d_t(
            H_HS=spec_HS,
            H_MR=spec_MR,
            H_OR=spec_OR,
            A_A=A_A,
            A_MR=A_MR,
            A_OR=A_OR,
            region=region,
            mode_MR=mode_MR,
            mode_OR=mode_OR,
            L_T_H_rad=L_T_H_d_t,
            CG=CG)
    else:
        return calc_E_K_d_t(1, spec_MR, A_A, A_MR, A_OR, L_T_H_d_t[0])

# 居室のみを暖房する方式における日付の時刻における 1 時間当たりの主たる居室に設置された暖房設備機器のその他の一次エネルギー (9d)
def get_E_M_H_hs_MR_d_t(A_A, A_MR, A_OR, spec_MR, spec_HS, L_H_d_t):
    if spec_MR['type'] in ['温水暖房用パネルラジエーター', '温水暖房用床暖房', '温水暖房用ファンコンベクター']:
        return hwh.get_E_M_H_d_t(spec_HS)
    else:
        return calc_E_M_d_t(1, spec_MR, A_A, A_MR, A_OR, L_H_d_t[0])

$$
\large
E_{E,H,hs,MR,d,t} = E_{E,H,hs,d,t,i}|_{i=1}
$$

$$
\large
E_{G,H,hs,MR,d,t} = E_{G,H,hs,d,t,i}|_{i=1}
$$

$$
\large
E_{K,H,hs,MR,d,t} = E_{K,H,hs,d,t,i}|_{i=1}
$$

$$
\large
E_{M,H,hs,MR,d,t} = E_{M,H,hs,d,t,i}|_{i=1}
$$

$E_{E,H,hs,d,t,i}$：日付$d$の時刻$t$における1時間当たりの暖冷房区画$i$に設置された暖房設備機器の消費電力量(kWh/h)  
$E_{G,H,hs,d,t,i}$：日付$d$の時刻$t$における1時間当たりの暖冷房区画$i$に設置された暖房設備機器のガス消費量(MJ/h)  
$E_{K,H,hs,d,t,i}$：日付$d$の時刻$t$における1時間当たりの暖冷房区画$i$に設置された暖房設備機器の灯油消費量(MJ/h)  
$E_{M,H,hs,d,t,i}$：日付$d$の時刻$t$における1時間当たりの暖冷房区画$i$に設置された暖房設備機器のその他の燃料による一次エネルギー消費量(MJ/h)  

$$
\large
E_{E,H,hs,OR,d,t} = \sum_{i=2～5} E_{E,H,hs,d,t,i} 
$$

$$
\large
E_{G,H,hs,OR,d,t} = \sum_{i=2～5} E_{G,H,hs,d,t,i} 
$$

$$
\large
E_{K,H,hs,OR,d,t} = \sum_{i=2～5} E_{K,H,hs,d,t,i} 
$$

$$
\large
E_{M,H,hs,OR,d,t} = \sum_{i=2～5} E_{M,H,hs,d,t,i} 
$$

In [ ]:
# (11a)
def get_E_E_H_hs_OR_d_t(region, A_A, A_MR, A_OR, mode_MR, mode_OR, spec_MR, spec_OR, spec_HS, L_T_H_d_t_i, CG=None):
    # その他の居室がない場合
    if spec_OR is None:
        return np.zeros(24 * 365)
    else:
        if spec_OR['type'] in ['温水暖房用パネルラジエーター', '温水暖房用床暖房', '温水暖房用ファンコンベクター']:
            return hwh.get_E_E_H_d_t(
                H_HS=spec_HS,
                H_MR=spec_MR,
                H_OR=spec_OR,
                A_A=A_A,
                A_MR=A_MR,
                A_OR=A_OR,
                region=region,
                mode_MR=mode_MR,
                mode_OR=mode_OR,
                L_T_H_rad=L_T_H_d_t_i,
                CG=CG)
        else:
            # 暖房区画i=2～5の電力消費量を計算して合算する
            return np.sum(
                [calc_E_E_H_d_t(i, spec_OR, A_A, A_MR, A_OR, region, mode_OR, L_T_H_d_t_i[i - 1]) for i in
                 [3, 4, 5]], axis=0)

# (11b)
def get_E_G_H_hs_OR_d_t(region, A_A, A_MR, A_OR, mode_MR, mode_OR, spec_MR, spec_OR, spec_HS, L_T_H_d_t, CG=None):
    # その他の居室がない場合
    if spec_OR is None:
        return np.zeros(24 * 365)

    else:
        if spec_OR['type'] in ['温水暖房用パネルラジエーター', '温水暖房用床暖房', '温水暖房用ファンコンベクター']:
            return hwh.get_E_G_H_d_t(
                H_HS=spec_HS,
                H_MR=spec_MR,
                H_OR=spec_OR,
                A_A=A_A,
                A_MR=A_MR,
                A_OR=A_OR,
                region=region,
                mode_MR=mode_MR,
                mode_OR=mode_OR,
                L_T_H_rad=L_T_H_d_t,
                CG=CG)
        else:
            return np.sum(
                [calc_E_G_H_d_t(i, spec_OR, A_A, A_MR, A_OR, L_T_H_d_t[i - 1])
                 for i in
                 range(2, 6)], axis=0)

# 居室のみを暖房する方式における日付の時刻における 1 時間当たりのその他の居室に設置された暖房設備機器のガス消費量 (11c)
def get_E_K_H_hs_OR_d_t(region, A_A, A_MR, A_OR, spec_MR, spec_OR, spec_HS, mode_MR, mode_OR, L_T_H_d_t, CG):
    # その他の居室がない場合
    if spec_OR is None:
        return np.zeros(24 * 365)

    else:
        if spec_OR['type'] in ['温水暖房用パネルラジエーター', '温水暖房用床暖房', '温水暖房用ファンコンベクター']:
            return hwh.get_E_K_H_d_t(
                H_HS=spec_HS,
                H_MR=spec_MR,
                H_OR=spec_OR,
                A_A=A_A,
                A_MR=A_MR,
                A_OR=A_OR,
                region=region,
                mode_MR=mode_MR,
                mode_OR=mode_OR,
                L_T_H_rad=L_T_H_d_t,
                CG=CG)
        else:
            return np.sum(
                [calc_E_K_d_t(i, spec_OR, A_A, A_MR, A_OR, L_T_H_d_t[i - 1])
                 for i in
                 range(2, 6)], axis=0)

# 居室のみを暖房する方式における日付の時刻における 1 時間当たりのその他の居室に設置された暖房設備機器のその他の一次エネルギー (11d)
def get_E_M_H_hs_OR_d_t(A_A, A_MR, A_OR, spec_OR, spec_HS, L_H_d_t):
    # その他の居室がない場合
    if spec_OR is None:
        return np.zeros(24 * 365)

    else:

        if spec_OR['type'] in ['温水暖房用パネルラジエーター', '温水暖房用床暖房', '温水暖房用ファンコンベクター']:
            return hwh.get_E_M_H_d_t(spec_HS)
        else:
            return np.sum(
                [calc_E_M_d_t(i, spec_OR, A_A, A_MR, A_OR, L_H_d_t[i - 1]) for
                 i in
                 range(2, 6)], axis=0)


# 暖房区画1つと1つの暖房設備機器によって消費されるその他の一次エネルギー消費量
def calc_E_M_d_t(i, device, A_A, A_MR, A_OR, L_H_d_t):
    # 床面積
    A_HCZ = get_A_HCZ_i(i, A_A, A_MR, A_OR)

    if device['type'] == 'ルームエアコンディショナー':
        # その他の一次エネルギー消費量の計算
        E_M_d_t = rac.get_E_M_H_d_t()

    elif device['type'] == 'FF暖房機':
        # その他の一次エネルギー消費量の計算
        E_M_d_t = ff.get_E_M_H_d_t()

    elif device['type'] == '電気蓄熱暖房器':
        # その他の一次エネルギー消費量の計算
        E_M_d_t = ets.get_E_M_H_d_t()

    elif device['type'] == '電気ヒーター床暖房':
        # その他の一次エネルギー消費量の計算
        E_M_d_t = eheater.get_E_M_H_d_t()

    elif device['type'] in ['温水暖房用パネルラジエーター', '温水暖房用床暖房', '温水暖房用ファンコンベクター']:
        # 温水暖房は第7節で取り扱うためここでは処理しない
        raise UserWarning(device['type'])

    elif device['type'] == 'ルームエアコンディショナー付温水床暖房機':
        # その他の一次エネルギー消費量の計算
        E_M_d_t = racfh.get_E_M_d_t()

    else:
        raise ValueError(device['type'])

    return E_M_d_t

#### 6.4 暖房設備の未処理暖房負荷の設計一次エネルギー消費量相当値 

#### 6.4.1 住戸全体を連続的に暖房する方式

$$
\large
E_{UT,H,d,t} = Q_{UT,H,A,d,t} \times \alpha_{UT,H,A}
$$

$E_{UT,H,d,t}$：日付$d$の時刻$t$における1時間当たりの暖房設備の未処理暖房負荷の設計一次エネルギー消費量相当値(MJ/h)  
$Q_{UT,H,A,d,t}$：住戸全体を連続的に暖房する方式における日付$d$の時刻$t$における1時間当たりの暖房設備の未処理暖房負荷(MJ/h)  
$\alpha_{UT,H,A}$：住戸全体を連続的に暖房する方式における暖房設備の未処理暖房負荷を未処理暖房負荷の設計一次エネルギー消費量相当値に換算するための係数  

In [ ]:
# 暖房設備の未処理暖房負荷の設計一次エネルギー消費量相当値 (12)
def get_E_UT_H_d_t__modeA(region, A_A, H_A, L_H_A_d_t):
    # 未処理暖房負荷を未処理暖房負荷の設計一次エネルギー消費量相当値に換算するための係数
    alpha_UT_H_A = get_alpha_UT_H_A(region)

    # 仕様の取得
    if 'q_rtd_H' in H_A and 'P_rtd_H' in H_A:
        q_rtd_H = H_A['q_rtd_H']
    else:
        q_rtd_H = dc_spec.get_q_rtd_H(region, A_A)

    # 未処理負荷を取得
    Q_UT_H_A_d_t = dc.get_Q_UT_H_d_t(
        region=region,
        q_rtd_H=q_rtd_H,
        L_H_d_t=L_H_A_d_t
    )

    return Q_UT_H_A_d_t * alpha_UT_H_A

<div style="text-align: center;font-weight: bold;">表　未処理暖房負荷を未処理暖房負荷の設計一次エネルギー消費量相当値に換算するための係数</div>

|地域の区分|住戸全体を連続的に暖房する方式|
|-|-|
|1|1.61|
|2|1.46|
|3|1.32|
|4|1.30|
|5|1.20|
|6|1.09|
|7|1.12|
|8|‐|

In [ ]:
# 表 3 未処理暖房負荷を未処理暖房負荷の設計一次エネルギー消費量相当値に換算するための係数
table_3 = [
    1.61,
    1.46,
    1.32,
    1.30,
    1.20,
    1.09,
    1.12
]

In [ ]:
# 未処理暖房負荷を未処理暖房負荷の設計一次エネルギー消費量相当値に換算するための係数
def get_alpha_UT_H_A(region):
    return table_3[region - 1]

#### 6.4.2 居室のみを暖房する方式

$$
\large
E_{UT,H,d,t} = Q_{UT,H,MR,d,t} \times \alpha_{UT,H,MR} + Q_{UT,H,OR,d,t} \times \alpha_{UT,H,OR}
$$

$E_{UT,H,d,t}$：日付$d$の時刻$t$における1時間当たりの暖房設備の未処理暖房負荷の設計一次エネルギー消費量相当値(MJ/h)  
$Q_{UT,H,MR,d,t}$：居室のみを暖房する方式における日付$d$の時刻$t$における1時間当たりの主たる居室に設置された暖房設備の未処理暖房負荷(MJ/h)  
$Q_{UT,H,OR,d,t}$：居室のみを暖房する方式における日付$d$の時刻$t$における1時間当たりのその他の居室に設置された暖房設備の未処理暖房負荷(MJ/h)  
$\alpha_{UT,H,MR}$：居室のみを暖房する方式における主たる居室に設置された暖房設備の未処理暖房負荷を未処理暖房負荷の設計一次エネルギー消費量相当値に換算するための係数  
$\alpha_{UT,H,OR}$：居室のみを暖房する方式におけるその他の居室に設置された暖房設備の未処理暖房負荷を未処理暖房負荷の設計一次エネルギー消費量相当値に換算するための係数  

<div style="text-align: center;font-weight: bold;">表　未処理暖房負荷を未処理暖房負荷の設計一次エネルギー消費量相当値に換算するための係数</div>

<table>
<thead>
<tr>
<th rowspan="3">地域の区分</th>
<th colspan="4">居室のみを暖房する方式</th>
</tr>
<tr>
<th colspan="2">主たる居室</th>
<th colspan="2">その他の居室</th>
</tr>
<tr>
<th>連続運転</th>
<th>間歇運転</th>
<th>連続運転</th>
<th>間歇運転</th>
</tr>
</thead>
<tbody>
<tr><th>1</th><td>1.59</td><td>1.21</td><td>1.59</td><td>1.22</td></tr>
<tr><th>2</th><td>1.66</td><td>1.22</td><td>1.66</td><td>1.24</td></tr>
<tr><th>3</th><td>1.63</td><td>1.22</td><td>1.63</td><td>1.23</td></tr>
<tr><th>4</th><td>1.60</td><td>1.21</td><td>1.60</td><td>1.23</td></tr>
<tr><th>5</th><td>1.53</td><td>1.05</td><td>1.53</td><td>1.04</td></tr>
<tr><th>6</th><td>1.57</td><td>0.96</td><td>1.57</td><td>1.00</td></tr>
<tr><th>7</th><td>1.63</td><td>1.01</td><td>1.63</td><td>1.34</td></tr>
<tr><th>8</th><td>-</td><td>-</td><td>-</td><td>-</td></tr>
</tbody>
</table>

In [ ]:
# 表 4 未処理暖房負荷を未処理暖房負荷の設計一次エネルギー消費量相当値に換算するための係数
table_4 = [
    (1.59, 1.21, 1.59, 1.22),
    (1.66, 1.22, 1.66, 1.24),
    (1.63, 1.22, 1.63, 1.23),
    (1.60, 1.21, 1.60, 1.23),
    (1.53, 1.05, 1.53, 1.04),
    (1.57, 0.96, 1.57, 1.00),
    (1.63, 1.01, 1.63, 1.34)
]

In [ ]:
# 居室のみを暖房する方式における主たる居室に設置された暖房設備の未処理暖房負荷を未処理暖房負荷の設計一次エネルギー消費量相当値に換算するための係数
def get_alpha_UT_H_MR(region, mode_H_MR):
    if mode_H_MR in ['ろ', '連続運転']:
        return table_4[region - 1][0]
    elif mode_H_MR in ['は', '間歇運転']:
        return table_4[region - 1][1]
    else:
        raise ValueError(mode_H_MR)


# 居室のみを暖房する方式におけるその他の居室に設置された暖房設備の未処理暖房負荷を未処理暖房負荷の設計一次エネルギー消費量相当値に換算するための係数
def get_alpha_UT_H_OR(region, mode_H_OR):
    if mode_H_OR in ['ろ', '連続運転']:
        return table_4[region - 1][0]
    elif mode_H_OR in ['は', '間歇運転']:
        return table_4[region - 1][3]
    else:
        raise ValueError(mode_H_OR)


#### 6.5 コージェネレーション設備又は住棟セントラル暖房設備が賄う温水暖房の熱負荷等

省略

### 7．冷房設備の一次エネルギー消費量及び処理負荷と未処理負荷

#### 7.1 処理負荷及び未処理負荷

#### 7.1.1 住戸全体を連続的に冷房する方式

$$
\large
Q_{T,CS,A,d,t} = min(Q_{max,CS,d,t}, L_{CS,A,d,t})
$$

$$
\large
Q_{UT,CS,A,d,t} = L_{CS,A,d,t} - Q_{T,CS,A,d,t}
$$

$Q_{T,CS,A,d,t}$：住戸全体を連続的に冷房する方式における日付$d$の時刻$t$における1時間当たりの冷房設備の処理冷房顕熱負荷(MJ/h)  
$Q_{UT,CS,A,d,t}$：住戸全体を連続的に冷房する方式における日付$d$の時刻$t$における1時間当たりの冷房設備の未処理冷房顕熱負荷(MJ/h)  
$Q_{max,CS,d,t}$：日付$d$の時刻$t$における1時間当たりの冷房設備機器の最大冷房顕熱出力(MJ/h)  
$L_{CS,A,d,t}$：住戸全体を連続的に冷房する方式における日付$d$の時刻$t$における1時間当たりの冷房顕熱負荷の合計(MJ/h)

In [ ]:
# (14a)
def get_Q_T_CS_A_d_t(Q_max_CS_d_t, L_CS_A_d_t):
    return np.min([Q_max_CS_d_t, L_CS_A_d_t], axis=0)


# (14b)
def get_Q_UT_CS_A_d_t(L_CS_A_d_t, Q_T_CS_A_d_t):
    return L_CS_A_d_t - Q_T_CS_A_d_t

$$
\large
Q_{T,CL,A,d,t} = min(Q_{max,CL,d,t}, L_{CL,A,d,t})
$$

$$
\large
Q_{UT,CL,A,d,t} = L_{CL,A,d,t} - Q_{T,CL,A,d,t}
$$

$Q_{T,CL,A,d,t}$：住戸全体を連続的に冷房する方式における日付$d$の時刻$t$における1時間当たりの冷房設備の処理冷房潜熱負荷(MJ/h)  
$Q_{UT,CL,A,d,t}$：住戸全体を連続的に冷房する方式における日付$d$の時刻$t$における1時間当たりの冷房設備の未処理冷房潜熱負荷(MJ/h)  
$Q_{max,C,d,tL}$：日付$d$の時刻$t$における1時間当たりの冷房設備機器の最大冷房潜熱出力(MJ/h)  
$L_{CL,A,d,t}$：住戸全体を連続的に冷房する方式における日付$d$の時刻$t$における1時間当たりの冷房潜熱負荷の合計(MJ/h)

$L_{CS,A,d,t}$が0未満の場合は0とする。$L_{CL,A,d,t}$が0未満の場合は0とする。 

In [ ]:
# (15a)
def get_Q_T_CL_A_d_t(Q_max_CL_d_t, L_CL_A_d_t):
    return np.min([Q_max_CL_d_t, L_CL_A_d_t], axis=0)


# (15b)
def get_Q_UT_CL_A_d_t(L_CL_A_d_t, Q_T_CL_A_d_t):
    return L_CL_A_d_t - Q_T_CL_A_d_t

$$
\large
L_{CS,A,d,t} = \sum_{i=1～12} L_{CS,d,t,i} 
$$

$$
\large
L_{CL,A,d,t} = \sum_{i=1～12} L_{CL,d,t,i} 
$$

$L_{CS,d,t,i}$：日付$d$の時刻$t$における暖冷房区画$i$の1時間当たりの冷房顕熱負荷(MJ/h)  
$L_{CL,d,t,i}$：日付$d$の時刻$t$における暖冷房区画$i$の1時間当たりの冷房潜熱負荷(MJ/h)  

In [ ]:
# 住戸全体を連続的に冷房する方式における1時間当たりの冷房顕熱負荷の合計 (16a)
def get_L_CS_A_d_t(region, A_A, A_MR, A_OR, Q, mu_C, NV_MR, NV_OR, r_A_ufvnt, underfloor_insulation):
    # i=1～12の冷房区画の冷房顕熱負荷を計算して合算
    mode = ['全館連続'] * 12
    L_CS_A_d_t = np.sum(ld.get_L_CS_d_t_i(
        region=region,
        A_A=A_A,
        A_MR=A_MR,
        A_OR=A_OR,
        mode=mode,
        NV_MR=NV_MR,
        NV_OR=NV_OR,
        Q=Q,
        mu_C=mu_C,
        r_A_ufvnt=r_A_ufvnt,
        underfloor_insulation=underfloor_insulation,
    ), axis=0)

    L_CS_A_d_t[L_CS_A_d_t < 0] = 0

    return L_CS_A_d_t


# 住戸全体を連続的に冷房する方式における1時間当たりの冷房潜熱負荷の合計 (16b)
def get_L_CL_A_d_t(region, A_A, A_MR, A_OR, Q, mu_C, NV_MR, NV_OR):
    # i=1～12の冷房区画の冷房潜熱負荷を計算して合算
    mode = ['全館連続'] * 12
    L_CL_A_d_t = np.sum(ld.get_L_CL_d_t_i(
        region=region,
        A_A=A_A,
        A_MR=A_MR,
        A_OR=A_OR,
        mode=mode,
        NV_MR=NV_MR,
        NV_OR=NV_OR,
        Q=Q,
        mu_C=mu_C
    ), axis=0)

    L_CL_A_d_t[L_CL_A_d_t < 0] = 0

    return L_CL_A_d_t

#### 7.1.2 居室のみを冷房する方式

$$
\large
Q_{T,CS,MR,d,t} = Q_{T,CS,d,t,i}|_{i=1}
$$

$$
\large
Q_{T,CL,MR,d,t} = Q_{T,CL,d,t,i}|_{i=1}
$$

$$
\large
Q_{UT,CS,MR,d,t} = Q_{UT,CS,d,t,i}|_{i=1}
$$

$$
\large
Q_{UT,CL,MR,d,t} = Q_{UT,CL,d,t,i}|_{i=1}
$$

$Q_{T,CS,MR,d,t}$：居室のみを冷房する方式における日付$d$の時刻$t$における1時間当たりの主たる居室に設置された冷房設備の処理冷房顕熱負荷(MJ/h)  
$Q_{T,CL,MR,d,t}$：居室のみを冷房する方式における日付$d$の時刻$t$における1時間当たりの主たる居室に設置された冷房設備の処理冷房潜熱負荷(MJ/h)  
$Q_{UT,CS,MR,d,t}$：居室のみを冷房する方式における日付$d$の時刻$t$における1時間当たりの主たる居室に設置された冷房設備の未処理冷房顕熱負荷(MJ/h)  
$Q_{UT,CL,MR,d,t}$：居室のみを冷房する方式における日付$d$の時刻$t$における1時間当たりの主たる居室に設置された冷房設備の未処理冷房潜熱負荷(MJ/h)  
$Q_{T,CS,d,t,i}$：日付$d$の時刻$t$における1時間当たりの暖冷房区画$i$に設置された冷房設備機器の処理冷房顕熱負荷(MJ/h)  
$Q_{T,CL,d,t,i}$：日付$d$の時刻$t$における1時間当たりの暖冷房区画$i$に設置された冷房設備機器の処理冷房潜熱負荷(MJ/h)  
$Q_{UT,CS,d,t,i}$：日付$d$の時刻$t$における1時間当たりの暖冷房区画$i$に設置された冷房設備機器の未処理冷房顕熱負荷(MJ/h)  
$Q_{UT,CL,d,t,i}$：日付$d$の時刻$t$における1時間当たりの暖冷房区画$i$に設置された冷房設備機器の未処理冷房潜熱負荷(MJ/h)

In [ ]:
# 居室のみを冷房する方式における日付の時刻における1時間当たりの主たる居室に設置された冷房設備の処理冷房顕熱負荷 (17a)
def get_Q_T_CS_MR_d_t(**args):
    # i=1の冷房区画の処理冷房顕熱負荷を計算して合算
    return get_Q_T_CS_d_t_i(i=1, **args)


# 居室のみを冷房する方式における日付の時刻における1時間当たりの主たる居室に設置された冷房設備の処理冷房潜熱負荷 (17b)
def get_Q_T_CL_MR_d_t(**args):
    # i=1の冷房区画の処理冷房潜熱負荷を計算して合算
    return get_Q_T_CL_d_t_i(i=1, **args)


# 居室のみを冷房する方式における日付の時刻における1時間当たりの主たる居室に設置された冷房設備の未処理処理冷房顕熱負荷 (17c)
def get_Q_UT_CS_MR_d_t(**args):
    # i=1の冷房区画の未処理冷房顕熱負荷を計算して合算
    return get_Q_UT_CS_d_t_i(i=1, **args)


# 居室のみを冷房する方式における日付の時刻における1時間当たりの主たる居室に設置された冷房設備の未処理処理冷房潜熱負荷 (17d)
def get_Q_UT_CL_MR_d_t(**args):
    # i=1の冷房区画の未処理冷房潜熱負荷を計算して合算
    return get_Q_UT_CL_d_t_i(i=1, **args)

$$
\large
Q_{T,CS,OR,d,t} = \sum_{i=2～5} Q_{T,CS,d,t,i} 
$$

$$
\large
Q_{T,CL,OR,d,t} = \sum_{i=2～5} Q_{T,CL,d,t,i} 
$$

$$
\large
Q_{UT,CS,OR,d,t} = \sum_{i=2～5} Q_{UT,CS,d,t,i} 
$$

$$
\large
Q_{UT,CL,OR,d,t} = \sum_{i=2～5} Q_{UT,CL,d,t,i} 
$$

$Q_{T,CS,OR,d,t}$：居室のみを冷房する方式における日付$d$の時刻$t$における1時間当たりのその他の居室に設置された冷房設備の処理冷房顕熱負荷(MJ/h)  
$Q_{T,CL,OR,d,t}$：居室のみを冷房する方式における日付$d$の時刻$t$における1時間当たりのその他の居室に設置された冷房設備の処理冷房潜熱負荷(MJ/h)  
$Q_{UT,CS,OR,d,t}$：居室のみを冷房する方式における日付$d$の時刻$t$における1時間当たりのその他の居室に設置された冷房設備の未処理冷房顕熱負荷(MJ/h)  
$Q_{UT,CL,OR,d,t}$：居室のみを冷房する方式における日付$d$の時刻$t$における1時間当たりのその他の居室に設置された冷房設備の未処理冷房潜熱負荷(MJ/h)  

In [ ]:
# 居室のみを冷房する方式における日付の時刻における1時間当たりのその他の居室に設置された冷房設備の処理冷房顕熱負荷 (18a)
def get_Q_T_CS_OR_d_t(**args):
    # i=2～5の冷房区画の処理冷房顕熱負荷を計算して合算
    return np.sum([get_Q_T_CS_d_t_i(i, **args) for i in range(2, 6)], axis=0)


# 居室のみを冷房する方式における日付の時刻における1時間当たりのその他の居室に設置された冷房設備の処理冷房潜熱負荷 (18b)
def get_Q_T_CL_OR_d_t(**args):
    # i=2～5の冷房区画の処理冷房潜熱負荷を計算して合算
    return np.sum([get_Q_T_CL_d_t_i(i, **args) for i in range(2, 6)], axis=0)


# 居室のみを冷房する方式における日付の時刻における1時間当たりのその他の居室に設置された冷房設備の未処理冷房顕熱負荷 (18c)
def get_Q_UT_CS_OR_d_t(**args):
    # i=2～5の冷房区画の未処理冷房顕熱負荷を計算して合算
    return np.sum([get_Q_UT_CS_d_t_i(i, **args) for i in range(2, 6)], axis=0)


# 居室のみを冷房する方式における日付の時刻における1時間当たりのその他の居室に設置された冷房設備の未処理冷房潜熱負荷 (18d)
def get_Q_UT_CL_OR_d_t(**args):
    # i=2～5の冷房区画の未処理冷房潜熱負荷を計算して合算
    return np.sum([get_Q_UT_CL_d_t_i(i, **args) for i in range(2, 6)], axis=0)

$$
\large
Q_{T,CS,d,t,i} = min(Q_{max,CS,d,t,i}, L_{CS,d,t,i})
$$

$$
\large
Q_{UT,CS,d,t,i} = L_{CS,d,t,i} - Q_{T,CS,d,t,i}
$$

$Q_{max,CS,d,t,i}$：日付$d$の時刻$t$における1時間当たりの暖冷房区画$i$に設置された冷房設備機器の最大冷房顕熱出力(MJ/h)  
$L_{CS,d,t,i}$：日付$d$の時刻$t$における暖冷房区画$i$の1時間当たりの冷房顕熱負荷(MJ/h)  

In [ ]:
# (19a)
def get_Q_T_CS_d_t_i(Q_max_CS_d_t_i, L_CS_d_t_i):
    return np.min([Q_max_CS_d_t_i, L_CS_d_t_i], axis=0)


# (19b)
def get_Q_UT_CS_d_t_i(L_CS_d_t_i, Q_T_CS_d_t_i):
    return L_CS_d_t_i - Q_T_CS_d_t_i

$$
\large
Q_{T,CL,d,t,i} = min(Q_{max,CL,d,t,i}, L_{CL,d,t,i})
$$

$$
\large
Q_{UT,CL,d,t,i} = L_{CL,d,t,i} - Q_{T,CL,d,t,i}
$$

$Q_{max,CL,d,t,i}$：日付$d$の時刻$t$における1時間当たりの暖冷房区画$i$に設置された冷房設備機器の最大冷房潜熱出力(MJ/h)  
$L_{CL,d,t,i}$：日付$d$の時刻$t$における暖冷房区画$i$の1時間当たりの冷房潜熱負荷(MJ/h)  

$L_{CS,d,t,i}$が0未満の場合は0とする。また、$L_{CL,d,t,i}$が0未満の場合は0とする。 

In [ ]:
# (20a)
def get_Q_T_CL_d_t_i(Q_max_CL_d_t_i, L_CL_d_t_i):
    return np.min([Q_max_CL_d_t_i, L_CL_d_t_i], axis=0)


# (20b)
def get_Q_UT_CL_d_t_i(L_CL_d_t_i, Q_T_CL_d_t_i):
    return L_CL_d_t_i - Q_T_CL_d_t_i

#### 7.2 冷房設備のエネルギー消費量

$$
\large
E_{E,C,d,t} = E_{E,C,hs,d,t}
$$

$$
\large
E_{G,C,d,t} = E_{G,C,hs,d,t}
$$

$$
\large
E_{K,C,d,t} = E_{K,C,hs,d,t}
$$

$$
\large
E_{M,C,d,t} = E_{M,C,hs,d,t}
$$

$E_{E,C,d,t}$：日付$d$の時刻$t$における1時間当たりの冷房設備の消費電力量(kWh/h)  
$E_{G,C,d,t}$：日付$d$の時刻$t$における1時間当たりの冷房設備のガス消費量(MJ/h)  
$E_{K,C,d,t}$：日付$d$の時刻$t$における1時間当たりの冷房設備の灯油消費量(MJ/h)  
$E_{M,C,d,t}$：日付$d$の時刻$t$における1時間当たりの冷房設備のその他の燃料による一次エネルギー消費量(MJ/h)  
$E_{E,C,hs,d,t}$：日付$d$の時刻$t$における1時間当たりの冷房設備機器の消費電力量(kWh/h)  
$E_{G,C,hs,d,t}$：日付$d$の時刻$t$における1時間当たりの冷房設備機器のガス消費量(MJ/h)  
$E_{K,C,hs,d,t}$：日付$d$の時刻$t$における1時間当たりの冷房設備機器の灯油消費量(MJ/h)  
$E_{M,C,hs,d,t}$：日付$d$の時刻$t$における1時間当たりの冷房設備機器のその他の燃料による一次エネルギー消費量(MJ/h)  

In [ ]:
# 1時間当たりの冷房設備の消費電力量 (21a)
def get_E_E_C_d_t(region, A_A, A_MR, A_OR, C_A=None, C_MR=None, C_OR=None, L_CS_A_d_t=None, L_CL_A_d_t=None,
                  L_CS_d_t=None, L_CL_d_t=None):
    return get_E_E_C_hs_d_t(region, A_A, A_MR, A_OR, C_A, C_MR, C_OR, L_CS_A_d_t, L_CL_A_d_t, L_CS_d_t, L_CL_d_t)


# 1時間当たりの冷房設備のガス消費量 (21b)
def get_E_G_C_d_t(**args):
    return get_E_G_C_hs_d_t()


# 1時間当たりの冷房設備の灯油消費量 (21c)
def get_E_K_C_d_t(**args):
    return get_E_K_C_hs_d_t()


# 1時間当たりの冷房設備のその他の一次エネルギー消費量 (21d)
def get_E_M_C_d_t(**args):
    return get_E_M_C_hs_d_t()

#### 7.3 冷房設備機器のエネルギー消費量

#### 7.3.1 住戸全体を連続的に冷房する方式

$$
\large
E_{E,C,hs,d,t} = E_{E,C,hs,A,d,t}
$$

$$
\large
E_{G,C,hs,d,t} = E_{G,C,hs,A,d,t}
$$

$$
\large
E_{K,C,hs,d,t} = E_{K,C,hs,A,d,t}
$$

$$
\large
E_{M,C,hs,d,t} = E_{M,C,hs,A,d,t}
$$

$E_{E,C,hs,d,t}$：日付$d$の時刻$t$における1時間当たりの冷房設備機器の消費電力量(kWh/h)  
$E_{G,C,hs,d,t}$：日付$d$の時刻$t$における1時間当たりの冷房設備機器のガス消費量(MJ/h)  
$E_{K,C,hs,d,t}$：日付$d$の時刻$t$における1時間当たりの冷房設備機器の灯油消費量(MJ/h)  
$E_{M,C,hs,d,t}$：日付$d$の時刻$t$における1時間当たりの冷房設備機器のその他の燃料による一次エネルギー消費量(MJ/h)  
$E_{E,C,hs,A,d,t}$：住戸全体を連続的に冷房する方式における日付$d$の時刻$t$における1時間当たりの冷房設備機器の消費電力量(kWh/h)  
$E_{G,C,hs,A,d,t}$：住戸全体を連続的に冷房する方式における日付$d$の時刻$t$における1時間当たりの冷房設備機器のガス消費量(MJ/h)  
$E_{K,C,hs,A,d,t}$：住戸全体を連続的に冷房する方式における日付$d$の時刻$t$における1時間当たりの冷房設備機器の灯油消費量(MJ/h)  
$E_{M,C,hs,A,d,t}$：住戸全体を連続的に冷房する方式における日付$d$の時刻$t$における1時間当たりの冷房設備機器のその他の燃料による一次エネルギー消費量(MJ/h)  

# 要確認：22aと23aが１つのコードにまとめられているが、このままでよいか

In [ ]:
# 1時間当たりの冷房設備機器の消費電力量 (22a)(23a)
def get_E_E_C_hs_d_t(region, A_A, A_MR, A_OR, C_A, C_MR, C_OR, L_CS_A_d_t, L_CL_A_d_t, L_CS_d_t, L_CL_d_t):
    if C_A is not None:
        # 仕様の取得
        if 'q_rtd_C' in C_A and 'P_rtd_C' in C_A:
            q_rtd_C = C_A['q_rtd_C']
            P_rtd_C = C_A['P_rtd_C']
        else:
            q_rtd_C = dc_spec.get_q_rtd_C(region, A_A)
            P_rtd_C = dc_spec.get_P_rtd_C(q_rtd_C)
        C_pl_C = C_A['C_pl_C']

        # 消費電力量の計算 (22a)
        E_E_C_d_t_i = dc.get_E_E_C_d_t(
            region=region,
            q_rtd_C=q_rtd_C,
            P_rtd_C=P_rtd_C,
            C_pl_C=C_pl_C,
            L_CS_d_t=L_CS_A_d_t,
            L_CL_d_t=L_CL_A_d_t
        )

        print('{} E_E_C_d_t_1 = {} [kWh] (L_H_d_t_1 = {} [MJ])'.format(C_A['type'], np.sum(E_E_C_d_t_i),
                                                                       np.sum(L_CS_A_d_t + L_CL_A_d_t)))
        return E_E_C_d_t_i
    elif C_MR is not None or C_OR is not None:
        # 消費電力量の計算 (23a)
        return get_E_E_C_hs_MR_d_t(region, A_A, A_MR, A_OR, C_MR, L_CS_d_t, L_CL_d_t) \
               + get_E_E_C_hs_OR_d_t(region, A_A, A_MR, A_OR, C_OR, L_CS_d_t, L_CL_d_t)
    else:
        return np.zeros(24*365)


# (22b)
def get_E_G_C_hs_d_t():
    return dc.get_E_G_C_hs_A_d_t()


# (22c)
def get_E_K_C_hs_d_t():
    return dc.get_E_K_C_hs_A_d_t()


# (22d)
def get_E_M_C_hs_d_t():
    return dc.get_E_M_C_hs_A_d_t()

#### 7.3.2 居室のみを冷房する方式

$$
\large
E_{E,C,hs,d,t} = E_{E,C,hs,MR,d,t} +  E_{E,C,hs,OR,d,t}
$$

$$
\large
E_{G,C,hs,d,t} = E_{G,C,hs,MR,d,t} +  E_{G,C,hs,OR,d,t}
$$

$$
\large
E_{K,C,hs,d,t} = E_{K,C,hs,MR,d,t} +  E_{K,C,hs,OR,d,t}
$$

$$
\large
E_{M,C,hs,d,t} = E_{M,C,hs,MR,d,t} +  E_{M,C,hs,OR,d,t}
$$

$E_{E,C,hs,d,t}$：日付$d$の時刻$t$における1時間当たりの冷房設備機器の消費電力量(kWh/h)  
$E_{G,C,hs,d,t}$：日付$d$の時刻$t$における1時間当たりの冷房設備機器のガス消費量(MJ/h)  
$E_{K,C,hs,d,t}$：日付$d$の時刻$t$における1時間当たりの冷房設備機器の灯油消費量(MJ/h)  
$E_{M,C,hs,d,t}$：日付$d$の時刻$t$における1時間当たりの冷房設備機器のその他の燃料による一次エネルギー消費量(MJ/h)  
$E_{E,C,hs,MR,d,t}$：居室のみを冷房する方式における日付$d$の時刻$t$における1時間当たりの主たる居室に設置された冷房設備機器の消費電力量(kWh/h)  
$E_{G,C,hs,MR,d,t}$：居室のみを冷房する方式における日付$d$の時刻$t$における1時間当たりの主たる居室に設置された冷房設備機器のガス消費量(MJ/h)  
$E_{K,C,hs,MR,d,t}$：居室のみを冷房する方式における日付$d$の時刻$t$における1時間当たりの主たる居室に設置された冷房設備機器の灯油消費量(MJ/h)  
$E_{M,C,hs,MR,d,t}$：居室のみを冷房する方式における日付$d$の時刻$t$における1時間当たりの主たる居室に設置された冷房設備機器のその他の燃料による一次エネルギー消費量(MJ/h)  
$E_{E,C,hs,OR,d,t}$：居室のみを冷房する方式における日付$d$の時刻$t$における1時間当たりのその他の居室に設置された冷房設備機器の消費電力量(kWh/h)  
$E_{G,C,hs,OR,d,t}$：居室のみを冷房する方式における日付$d$の時刻$t$における1時間当たりのその他の居室に設置された冷房設備機器のガス消費量(MJ/h)  
$E_{K,C,hs,OR,d,t}$：居室のみを冷房する方式における日付$d$の時刻$t$における1時間当たりのその他の居室に設置された冷房設備機器の灯油消費量(MJ/h)  
$E_{M,C,hs,OR,d,t}$：居室のみを冷房する方式における日付$d$の時刻$t$における1時間当たりのその他の居室に設置された冷房設備機器のその他の燃料による一次エネルギー消費量(MJ/h)  

In [ ]:
# (23b)
def get_E_G_C_hs_d_t():
    return get_E_G_C_hs_MR_d_t() + get_E_G_C_hs_OR_d_t()


# (23c)
def get_E_K_C_hs_d_t():
    return get_E_K_C_hs_MR_d_t() + get_E_K_C_hs_OR_d_t()


# (23d)
def get_E_M_C_hs_d_t():
    return get_E_M_C_hs_MR_d_t() + get_E_M_C_hs_OR_d_t()

$$
\large
E_{E,C,hs,MR,d,t} = E_{E,C,hs,d,t,i}|_{i=1}
$$

$$
\large
E_{G,C,hs,MR,d,t} = E_{G,C,hs,d,t,i}|_{i=1}
$$

$$
\large
E_{K,C,hs,MR,d,t} = E_{K,C,hs,d,t,i}|_{i=1}
$$

$$
\large
E_{M,C,hs,MR,d,t} = E_{M,C,hs,d,t,i}|_{i=1}
$$

$E_{E,C,hs,d,t,i}$：日付$d$の時刻$t$における1時間当たりの暖冷房区画$i$に設置された冷房設備機器の消費電力量(kWh/h)  
$E_{G,C,hs,d,t,i}$：日付$d$の時刻$t$における1時間当たりの暖冷房区画$i$に設置された冷房設備機器のガス消費量(MJ/h)  
$E_{K,C,hs,d,t,i}$：日付$d$の時刻$t$における1時間当たりの暖冷房区画$i$に設置された冷房設備機器の灯油消費量(MJ/h)  
$E_{M,C,hs,d,t,i}$：日付$d$の時刻$t$における1時間当たりの暖冷房区画$i$に設置された冷房設備機器のその他の燃料による一次エネルギー消費量(MJ/h)  

In [ ]:
# 居室のみを冷房する方式における、日付の時刻における1時間当たりの主たる居室に設置された冷房設備機器の消費電力 (24a)
def get_E_E_C_hs_MR_d_t(region, A_A, A_MR, A_OR, C_MR, L_CS_d_t, L_CL_d_t):
    spec_MR = C_MR

    if spec_MR['type'] == '設置しない' or spec_MR['type'] == 'その他':
        spec_MR = get_default_cooling_spec()

    E_E_C_d_t_i = calc_E_E_C_hs_d_t_i(1, spec_MR, region, A_A, A_MR, A_OR, L_CS_d_t, L_CL_d_t)

    return E_E_C_d_t_i


# 居室のみを冷房する方式における、日付の時刻における1時間当たりの主たる居室に設置された冷房設備機器のガス消費量 (24b)
def get_E_G_C_hs_MR_d_t():
    return rac.get_E_G_C_d_t()


# 居室のみを冷房する方式における、日付の時刻における1時間当たりの主たる居室に設置された冷房設備機器の灯油消費量 (24c)
def get_E_K_C_hs_MR_d_t():
    return rac.get_E_K_C_d_t()


# 居室のみを冷房する方式における、日付の時刻における1時間当たりの主たる居室に設置された冷房設備機器のその他の一次エネルギー消費量 (24d)
def get_E_M_C_hs_MR_d_t():
    return rac.get_E_M_C_d_t()

$$
\large
E_{E,C,hs,OR,d,t} = \sum_{i=2～5} E_{E,C,hs,d,t,i} 
$$

$$
\large
E_{G,C,hs,OR,d,t} = \sum_{i=2～5} E_{G,C,hs,d,t,i} 
$$

$$
\large
E_{K,C,hs,OR,d,t} = \sum_{i=2～5} E_{K,C,hs,d,t,i} 
$$

$$
\large
E_{M,C,hs,OR,d,t} = \sum_{i=2～5} E_{M,C,hs,d,t,i} 
$$

In [ ]:
# 居室のみを冷房する方式における、日付の時刻における1時間当たりのその他の居室に設置された冷房設備機器の消費電力 (25a)
def get_E_E_C_hs_OR_d_t(region, A_A, A_MR, A_OR, C_OR, L_CS_d_t, L_CL_d_t):
    # その他の居室がない場合
    if A_OR == 0:
        return np.zeros(24 * 365)

    spec_OR = C_OR

    if spec_OR['type'] == '設置しない' or spec_OR['type'] == 'その他':
        spec_OR = get_default_cooling_spec()

    tmp = []
    for i in [3, 4, 5]:
        E_E_C_d_t_i = calc_E_E_C_hs_d_t_i(i, spec_OR, region, A_A, A_MR, A_OR, L_CS_d_t, L_CL_d_t)
        tmp.append(E_E_C_d_t_i)

    return np.sum(tmp, axis=0)


def calc_E_E_C_hs_d_t_i(i, device, region, A_A, A_MR, A_OR, L_CS_d_t, L_CL_d_t):
    if device['type'] == 'ルームエアコンディショナー':

        # 仕様の取得
        A_HCZ_i = get_A_HCZ_i(i, A_A, A_MR, A_OR)
        q_rtd_C = rac_spec.get_q_rtd_C(A_HCZ_i)
        e_rtd_C = rac_spec.get_e_rtd_C(device['e_class'], q_rtd_C)

        # 電力消費量の計算
        E_E_C_d_t_i = rac.get_E_E_C_d_t(
            region=region,
            q_rtd_C=q_rtd_C,
            e_rtd_C=e_rtd_C,
            dualcompressor=device['dualcompressor'],
            L_CS_d_t=L_CS_d_t[i - 1],
            L_CL_d_t=L_CL_d_t[i - 1]
        )
    else:
        raise ValueError(device['type'])

    print('{} E_E_C_d_t_{} = {} [kWh] (L_H_d_t_{} = {} [MJ])'.format(device['type'], i, np.sum(E_E_C_d_t_i), i,
                                                                     np.sum(L_CS_d_t + L_CL_d_t)))

    return E_E_C_d_t_i


# 居室のみを冷房する方式における、日付の時刻における1時間当たりのその他の居室に設置された冷房設備機器のガス消費量 (25b)
def get_E_G_C_hs_OR_d_t():
    return np.sum([rac.get_E_G_C_d_t() for i in range(2, 6)], axis=0)


# 居室のみを冷房する方式における、日付の時刻における1時間当たりのその他の居室に設置された冷房設備機器の灯油消費量 (25c)
def get_E_K_C_hs_OR_d_t():
    return np.sum([rac.get_E_K_C_d_t() for i in range(2, 6)], axis=0)


# 居室のみを冷房する方式における、日付の時刻における1時間当たりのその他の居室に設置された冷房設備機器のその他の一次エネルギー消費量 (25d)
def get_E_M_C_hs_OR_d_t():
    return np.sum([rac.get_E_M_C_d_t() for i in range(2, 6)], axis=0)

## 付録 A 設置する暖房設備機器又は放熱器の種類に応じた暖房方式及び運転方法の決定方法並びに評価上想定される暖房設備機器又は放熱器の種類

### A.1 設置する暖房設備機器又は放熱器の種類に応じた暖房方式及び運転方法の決定方法 

<div style="text-align: center;font-weight: bold;">表　主たる居室及びその他の居室の運転方法（その他の居室がある場合）<br>（上段：主たる居室の運転方法 下段：その他の居室の運転方法） </div>

<table>
<tr>
<th colspan="2" rowspan="2"></th>
<th colspan="8">その他の居室に設置する暖冷房設備機器等</th>
</tr>
<tr>
<th>電気蓄熱<br>暖房器</th>
<th>パネル<br>ラジエーター</th>
<th>温水床暖房</th>
<th>ファン<br>コンベクター</th>
<th>ルーム<br>エアコン<br>ディショナー</th>
<th>FF暖房機</th>
<th>電気<br>ヒーター<br>床暖房</th>
<th>ルーム<br>エアコン<br>ディショナー付<br>温水床暖房</th>
</tr>

<tr>
<th rowspan="8">主たる居室に<br>設置する<br>暖冷房<br>設備機器等</th>
<th>電気蓄熱暖房器</th>
<td>連続<br>連続</td>
<td>連続<br>連続</td>
<td>連続<br>連続</td>
<td>連続<br>間歇</td>
<td>連続<br>間歇</td>
<td>連続<br>間歇</td>
<td>連続<br>間歇</td>
<td>連続<br>間歇</td>
</tr>
<tr>
<th>パネルラジエーター</th>
<td>連続<br>連続</td>
<td>連続<br>連続</td>
<td>連続<br>連続</td>
<td>連続<br>間歇</td>
<td>連続<br>間歇</td>
<td>連続<br>間歇</td>
<td>連続<br>間歇</td>
<td>連続<br>間歇</td>
</tr>
<tr>
<th>温水床暖房</th>
<td>連続<br>連続</td>
<td>連続<br>連続</td>
<td>連続<br>連続</td>
<td>間歇<br>間歇</td>
<td>間歇<br>間歇</td>
<td>間歇<br>間歇</td>
<td>間歇<br>間歇</td>
<td>間歇<br>間歇</td>
</tr>
<tr>
<th>ファンコンベクター</th>
<td>間歇<br>連続</td>
<td>間歇<br>連続</td>
<td>間歇<br>間歇</td>
<td>間歇<br>間歇</td>
<td>間歇<br>間歇</td>
<td>間歇<br>間歇</td>
<td>間歇<br>間歇</td>
<td>間歇<br>間歇</td>
</tr>
<tr>
<th>ルームエア<br>コンディショナー</th>
<td>間歇<br>連続</td>
<td>間歇<br>連続</td>
<td>間歇<br>間歇</td>
<td>間歇<br>間歇</td>
<td>間歇<br>間歇</td>
<td>間歇<br>間歇</td>
<td>間歇<br>間歇</td>
<td>間歇<br>間歇</td>
</tr>
<tr>
<th>FF 暖房機</th>
<td>間歇<br>連続</td>
<td>間歇<br>連続</td>
<td>間歇<br>間歇</td>
<td>間歇<br>間歇</td>
<td>間歇<br>間歇</td>
<td>間歇<br>間歇</td>
<td>間歇<br>間歇</td>
<td>間歇<br>間歇</td>
</tr>
<tr>
<th>電気ヒーター床暖房</th>
<td>間歇<br>連続</td>
<td>間歇<br>連続</td>
<td>間歇<br>間歇</td>
<td>間歇<br>間歇</td>
<td>間歇<br>間歇</td>
<td>間歇<br>間歇</td>
<td>間歇<br>間歇</td>
<td>間歇<br>間歇</td>
</tr>
<tr>
<th>ルーム<br>エアコン<br>ディショナー付<br>温水床暖房</th>
<td>間歇<br>連続</td>
<td>間歇<br>連続</td>
<td>間歇<br>間歇</td>
<td>間歇<br>間歇</td>
<td>間歇<br>間歇</td>
<td>間歇<br>間歇</td>
<td>間歇<br>間歇</td>
<td>間歇<br>間歇</td>
</tr>
</table>

<div style="text-align: center;font-weight: bold;">表　主たる居室の運転方法（その他の居室がない場合）</div>

<table>
<tr>
<th colspan="3"></th>
<th>運転方法</th>
</tr>
<tr>
<th rowspan="9">主たる居室に設置する<br>暖冷房設備機器等</th>
<th colspan="2">電気蓄熱暖房器</th>
<td>連続</td>
</tr>
<tr>
<th colspan="2">パネルラジエーター</th>
<td>連続</td>
</tr>
<tr>
<th rowspan="2">温水床暖房</th>
<th>1～2 地域</th>
<td>連続</td>
</tr>
<tr>
<th>3～7 地域</th>
<td>間歇</td>
</tr>
<tr>
<th colspan="2">ファンコンベクター</th>
<td>間歇</td>
</tr>
<tr>
<th colspan="2">ルームエアコンディショナー</th>
<td>間歇</td>
</tr>
<tr>
<th colspan="2">FF 暖房機</th>
<td>間歇</td>
</tr>
<tr>
<th colspan="2">電気ヒーター床暖房</th>
<td>間歇</td>
</tr>
<tr>
<th colspan="2">ルームエアコンディショナー付<br>温水床暖房</th>
<td>間歇</td>
</tr>
</table>

### A.2 複数の暖房設備機器等が設置される場合の評価上想定される暖房設備機器等の種類 

省略

### A.3 暖房設備機器等が設置されない場合の評価上想定される暖房設備機器等の種類

主たる居室若しくはその他の居室に暖房設備機器等を設置しない場合又は表 A.1 に掲げる暖房設備機器等以外の暖房設備機器等を設置する場合は、地域の区分に応じて表 A.5 に示す暖房設備機器等が設置されるものとして評価する。その際、パネルラジエーターが想定された場合（1 地域及び 2 地域）の配管の断熱措置については「断熱被覆のないもの」とし、温水暖房用熱源機については石油温水暖房機で、定格能力効率を0.830(83.0%)として評価する。また、FF 暖房機が想定された場合（3 地域及び 4 地域）の FF 暖房機の定格能力におけるエネルギー消費効率は0.860(86.0%)とする。ルームエアコンディショナーが想定された場合（5～7地域）のルームエアコンディショナーのエネルギー消費効率の区分は区分（ろ）とする。 

<div style="text-align: center;font-weight: bold;">表　主たる居室若しくはその他の居室に暖房設備機器等を設置しない場合又は表 A.1 に掲げる<br>暖房設備機器等以外の暖房設備機器等を設置する場合の評価において想定する暖房設備機器等</div>

<table>
<thead>
<tr>
	<th rowspan="2">地域の区分</th>
	<th colspan="2">評価において想定される暖房設備機器等</th>
</tr>
<tr>
	<th>主たる居室</th>
	<th>その他の居室</th>
</tr>
</thead>
<tbody>
<tr>
	<th>1</th>
	<td>パネルラジエーター</td>
	<td>パネルラジエーター</td>
</tr>
<tr>
	<th>2</th>
	<td>パネルラジエーター</td>
	<td>パネルラジエーター</td>
</tr>
<tr>
	<th>3</th>
	<td>ＦＦ暖房機</td>
	<td>ＦＦ暖房機</td>
</tr>
<tr>
	<th>4</th>
	<td>ＦＦ暖房機</td>
	<td>ＦＦ暖房機</td>
</tr>
<tr>
	<th>5</th>
	<td>ルームエアコンディショナー</td>
	<td>ルームエアコンディショナー</td>
</tr>
<tr>
	<th>6</th>
	<td>ルームエアコンディショナー</td>
	<td>ルームエアコンディショナー</td>
</tr>
<tr>
	<th>7</th>
	<td>ルームエアコンディショナー</td>
	<td>ルームエアコンディショナー</td>
</tr>
</tbody>
</table>

主たる居室若しくはその他の居室又はその両方に、温水床暖房、ファンコンベクター又はパネルラジエーターが設置された場合において、配管を設置しない場合においては、配管の断熱措置を「断熱被覆のないもの」として評価し、温水暖房用熱源機を設置しない場合又はその他の温水暖房用熱源機を設置する場合においては、表 A.6 に示す温水暖房用熱源機が設置されたものとして評価する。その場合の定格効率として、石油従来型温水暖房機の場合は0.830(83.0%)、ガス従来型温水暖房機の場合は0.825(82.5％)で評価する。

<div style="text-align: center;font-weight: bold;">表　温水暖房用熱源機を設置しない又はその他の温水暖房用熱源機を<br>設置する場合の評価において想定する温水暖房用熱源機</div>

<table>
<thead>
<tr>
	<th>地域の区分</th>
	<th>評価において想定される暖房設備機器等</th>
</tr>
</thead>
<tbody>
<tr>
	<th>1</th>
	<td>石油従来型温水暖房機</td>
</tr>
<tr>
	<th>2</th>
	<td>石油従来型温水暖房機</td>
</tr>
<tr>
	<th>3</th>
	<td>石油従来型温水暖房機</td>
</tr>
<tr>
	<th>4</th>
	<td>石油従来型温水暖房機</td>
</tr>
<tr>
	<th>5</th>
	<td>ガス従来型温水暖房機</td>
</tr>
<tr>
	<th>6</th>
	<td>ガス従来型温水暖房機</td>
</tr>
<tr>
	<th>7</th>
	<td>ガス従来型温水暖房機</td>
</tr>
</tbody>
</table>

## 付録 B 設置する冷房設備機器の種類に応じた冷房方式及び運転方法の決定方法並びに評価上想定される冷房設備機器の種類 

「居室のみを冷房する方式」の場合の主たる居室及びその他の居室における運転方法はともに間歇運転とする。  
主たる居室、その他の居室に冷房設備機器を設置しない場合又はルームエアコンディショナー以外の冷房設備機器を設置する場合は、ルームエアコンディショナーが設置されたものとして評価する。その際、ルームエアコンディショナーのエネルギー消費効率の区分は区分（ろ）とする。 